In [125]:
#Libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ttest_ind
import numpy as np
from scipy.stats import mannwhitneyu
from sklearn.model_selection import train_test_split
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression


## EDA

## Metadata:
sku: Stock Keeping Unit


In [2]:
# # Product Structure Sales
df1 = pd.read_csv("train/product_structures_sales.csv")
df1.head(100)

,structure_level_4,structure_level_3,structure_level_2,structure_level_1,sku,time_key,quantity
0,3020206,30202,302,3,3111,20230618,18.684000
1,3020608,30206,302,3,3278,20240731,396.100800
2,3020809,30208,302,3,3603,20230807,6.228000
3,3020608,30206,302,3,4604,20230131,27.403200
4,3040808,30408,304,3,3041,20230906,6.228000
...,...,...,...,...,...,...,...
95,3020809,30208,302,3,3603,20240522,11.210400
96,1010701,10107,101,1,3489,20231005,1259.201952
97,3020809,30208,302,3,3603,20240817,18.684000
98,3020608,30206,302,3,4604,20230914,4.982400


In [3]:
df1.dtypes

structure_level_4      int64
structure_level_3      int64
structure_level_2      int64
structure_level_1      int64
sku                    int64
time_key               int64
quantity             float64
dtype: object

In [4]:
df1.shape

(1864594, 7)

In [5]:
df1.isnull().sum()

structure_level_4    0
structure_level_3    0
structure_level_2    0
structure_level_1    0
sku                  0
time_key             0
quantity             0
dtype: int64

In [6]:
df1.duplicated().sum()

0

In [7]:
df1.describe()

,structure_level_4,structure_level_3,structure_level_2,structure_level_1,sku,time_key,quantity
count,1.864594e+06,1.864594e+06,1.864594e+06,1.864594e+06,1.864594e+06,1.864594e+06,1.864594e+06
mean,2.428731e+06,2.428724e+04,2.428015e+02,2.402365e+00,3.064254e+03,2.023519e+07,6.946481e+02
std,7.880273e+05,7.880271e+03,7.879493e+01,7.860546e-01,9.888995e+02,4.937769e+03,2.736481e+03
min,1.010101e+06,1.010100e+04,1.010000e+02,1.000000e+00,1.124000e+03,2.023010e+07,-2.341160e+04
25%,2.020506e+06,2.020500e+04,2.020000e+02,2.000000e+00,2.209000e+03,2.023062e+07,2.491200e+01
50%,3.020401e+06,3.020400e+04,3.020000e+02,3.000000e+00,3.085000e+03,2.023120e+07,1.108584e+02
75%,3.030709e+06,3.030700e+04,3.030000e+02,3.000000e+00,3.932000e+03,2.024052e+07,4.459248e+02
max,3.051115e+06,3.051100e+04,3.050000e+02,3.000000e+00,4.735000e+03,2.024103e+07,1.578543e+05


In [8]:
num_skus = df1['sku'].nunique()
print(f"You have {num_skus} different SKUs in your DataFrame.")



You have 3605 different SKUs in your DataFrame.


In [9]:
df1['time_key'] = pd.to_datetime(df1['time_key'], format='%Y%m%d')

In [10]:
# Check if there were any days where the total quantity sold was zero.

zero_quantity_dates = df1.groupby('time_key')['quantity'].sum().reset_index()

zero_quantity_dates = zero_quantity_dates[zero_quantity_dates['quantity'] == 0]

zero_quantity_dates


,time_key,quantity
356,2023-12-25,0.0
363,2024-01-01,0.0


In [12]:
# Remove lines where quantity was less than 0
df1 = df1[df1['quantity'] >= 0]


In [13]:
# Aggregate quantities by day and structure_level_4
df_daily = df1.groupby(['structure_level_4', 'time_key'], as_index=False)['quantity'].sum()
df_daily = df_daily.sort_values(['structure_level_4', 'time_key'])

# Calculate the compounded daily growth rate
df_daily['prev_quantity'] = df_daily.groupby('structure_level_4')['quantity'].shift(1)
df_daily['prev_date'] = df_daily.groupby('structure_level_4')['time_key'].shift(1)
df_daily['days_diff'] = (df_daily['time_key'] - df_daily['prev_date']).dt.days
df_daily['daily_growth_rate'] = (
    (df_daily['quantity'] / df_daily['prev_quantity']) ** (1 / df_daily['days_diff']) - 1
) * 100

# Remove invalid values
df_valid = df_daily.dropna(subset=['daily_growth_rate'])
df_valid = df_valid.replace([float('inf'), float('-inf')], pd.NA).dropna(subset=['daily_growth_rate'])

# Create 'month' column for monthly grouping
df_valid['month'] = df_valid['time_key'].dt.to_period('M').dt.to_timestamp()

# Calculate monthly median growth rate by structure_level_4
df_median_monthly = (
    df_valid.groupby(['structure_level_4', 'month'])['daily_growth_rate']
    .median()
    .reset_index()
)

# Plot line chart with Plotly
fig = px.line(
    df_median_monthly,
    x='month',
    y='daily_growth_rate',
    color='structure_level_4',
    title='Monthly Median of Compounded Daily Growth Rate (by structure_level_4)',
    labels={
        'month': 'Month',
        'daily_growth_rate': 'Median CDGR (%)',
        'structure_level_4': 'Structure Level 4'
    },
    template='plotly_dark'
)

fig.show()





In [14]:
#Calculate the overall median by structure_level_4
top10_structures = (
    df_median_monthly
    .groupby('structure_level_4')['daily_growth_rate']
    .median()
    .sort_values(ascending=False)
    .head(5)
    .index
)

# Filter the DataFrame to keep only the Top 10
df_top10 = df_median_monthly[df_median_monthly['structure_level_4'].isin(top10_structures)]

# Plot
fig = px.line(
    df_top10,
    x='month',
    y='daily_growth_rate',
    color='structure_level_4',
    title='Top 5 Structure Level 4 with Highest Median Daily Growth Rate (Monthly)',
    labels={
        'month': 'Month',
        'daily_growth_rate': 'Median of CDGR (%)',
        'structure_level_4': 'Structure Level 4'
    },
    template='plotly_white'
)

fig.show()




In [15]:
df_top10

,structure_level_4,month,daily_growth_rate
4444,2020808,2023-01-01,5.550266
4445,2020808,2023-02-01,11.314721
4446,2020808,2023-03-01,6.693712
4447,2020808,2023-04-01,14.338735
4448,2020808,2023-05-01,6.598985
...,...,...,...
5121,3011113,2024-06-01,10.630616
5122,3011113,2024-07-01,3.770863
5123,3011113,2024-08-01,3.223331
5124,3011113,2024-09-01,-0.057354


In [17]:
# Convert time_key to datetime
df1['date'] = pd.to_datetime(df1['time_key'], format='%Y%m%d')
df1['year'] = df1['date'].dt.year
df1['month'] = df1['date'].dt.month

# Get the top 5 structure_level_4 by total quantity
top5_levels = df1.groupby('structure_level_4')['quantity'].sum().nlargest(5).index
df_top = df1[df1['structure_level_4'].isin(top5_levels)]

# Group by structure_level_4, year, and month, and sum the quantities
monthly = df_top.groupby(['structure_level_4', 'year', 'month'])['quantity'].sum().reset_index()

# Bar chart for 2023 and 2024, organized into two rows

# Filter data for 2023
monthly_2023 = monthly[monthly['year'] == 2023]

# Filter data for 2024
monthly_2024 = monthly[monthly['year'] == 2024]

# Combine both DataFrames, adding a string column for the year
monthly_combined = pd.concat([
    monthly_2023.assign(year_str='2023'),
    monthly_2024.assign(year_str='2024')
])

# Define a fixed color palette for the top 5 structure_level_4
color_map = {
    3020206: '#1f77b4',  # Blue
    3020608: '#ff7f0e',  # Orange
    3020809: '#2ca02c',  # Green
    3040808: '#d62728',  # Red
    1010701: '#9467bd'   # Purple
}

# Create a bar chart with two rows (one per year) and columns per structure_level_4
fig = px.bar(
    monthly_combined,
    x='month',
    y='quantity',
    color='structure_level_4',
    facet_col='structure_level_4',  
    facet_row='year_str',  
    title='Monthly Quantity for Top 5 structure_level_4 (2023 and 2024)',
    labels={
        'month': 'Month',
        'quantity': 'Quantity',
        'structure_level_4': 'Structure (Level 4)',
        'year_str': 'Year'
    },
    barmode='group', 
    color_discrete_map=color_map  

)

fig.update_layout(
    xaxis=dict(tickmode='linear'),
    xaxis_tickangle=0,
    showlegend=True
)


fig.update_yaxes(tickformat='.0f')

fig.update_layout(coloraxis_showscale=False)

fig.show()




In [18]:
# Chain Campaigns
df2 = pd.read_csv ("train/chain_campaigns.csv")
df2.head()

,competitor,start_date,end_date,chain_campaign
0,chain,2024-08-12,2024-08-18,C2
1,competitorA,2023-09-22,2023-09-25,A2
2,chain,2024-09-23,2024-09-29,C1
3,chain,2024-04-08,2024-04-14,C1
4,competitorA,2023-10-20,2023-10-23,A2


In [19]:
df2.shape

(99, 4)

In [20]:
df2.describe()

,competitor,start_date,end_date,chain_campaign
count,99,99,99,99
unique,2,97,86,5
top,chain,2023-05-30,2024-01-21,C1
freq,53,2,2,30


In [21]:
df2.duplicated().sum()

0

In [22]:
df2.isnull().sum()

competitor        0
start_date        0
end_date          0
chain_campaign    0
dtype: int64

In [23]:
df2['start_date'] = pd.to_datetime(df2['start_date'])
df2['end_date'] = pd.to_datetime(df2['end_date'])
df2['duration_days'] = (df2['end_date'] - df2['start_date']).dt.days
df2['start_day'] = df2['start_date'].dt.day_name()

In [24]:
df2 = df2[df2['duration_days'] >= 0]

In [25]:
df2.describe()

,start_date,end_date,duration_days
count,98,98,98.000000
mean,2023-11-04 07:20:48.979591936,2023-11-09 01:28:09.795918336,4.755102
min,2023-01-03 00:00:00,2023-01-09 00:00:00,1.000000
25%,2023-06-01 06:00:00,2023-06-06 12:00:00,2.000000
50%,2023-10-18 12:00:00,2023-10-23 00:00:00,6.000000
75%,2024-03-01 00:00:00,2024-03-13 12:00:00,6.000000
max,2024-10-25 00:00:00,2024-10-27 00:00:00,30.000000
std,NaN,NaN,3.305919


In [26]:
df2.head(50)

,competitor,start_date,end_date,chain_campaign,duration_days,start_day
0,chain,2024-08-12,2024-08-18,C2,6,Monday
1,competitorA,2023-09-22,2023-09-25,A2,3,Friday
2,chain,2024-09-23,2024-09-29,C1,6,Monday
3,chain,2024-04-08,2024-04-14,C1,6,Monday
4,competitorA,2023-10-20,2023-10-23,A2,3,Friday
5,chain,2023-07-04,2023-07-10,C1,6,Tuesday
6,chain,2023-01-31,2023-02-06,C1,6,Tuesday
7,chain,2023-06-27,2023-07-03,C1,6,Tuesday
8,chain,2024-05-27,2024-06-02,C2,6,Monday
9,chain,2023-04-04,2023-04-10,C1,6,Tuesday


In [27]:
df2['competitor'].unique()

array(['chain', 'competitorA'], dtype=object)

In [28]:
# Sort by date
df_sorted = df2.sort_values('start_date').reset_index(drop=True)

# Add previous row information
df_sorted['prev_competitor'] = df_sorted['competitor'].shift(1)
df_sorted['prev_start_date'] = df_sorted['start_date'].shift(1)
df_sorted['prev_chain_campaign'] = df_sorted['chain_campaign'].shift(1)

# Calculate difference in days between current and previous start_date
df_sorted['days_from_prev'] = (df_sorted['start_date'] - df_sorted['prev_start_date']).dt.days

# Mark campaigns from competitorA that started within 7 days after a chain campaign
df_sorted['reactive_to_chain'] = (
    (df_sorted['competitor'] == 'competitorA') &
    (df_sorted['prev_competitor'] == 'chain') &
    (df_sorted['days_from_prev'] <= 7)
)

competitor_reactions = df_sorted[df_sorted['reactive_to_chain']][[
    'prev_chain_campaign', 'prev_start_date', 'chain_campaign', 'start_date', 'days_from_prev'
]]

# Mark chain campaigns that started within 7 days after competitorA's campaign
df_sorted['reactive_to_competitor'] = (
    (df_sorted['competitor'] == 'chain') &
    (df_sorted['prev_competitor'] == 'competitorA') &
    (df_sorted['days_from_prev'] <= 7)
)

chain_reactions = df_sorted[df_sorted['reactive_to_competitor']][[
    'prev_chain_campaign', 'prev_start_date', 'chain_campaign', 'start_date', 'days_from_prev'
]]

# Add reaction type column
df_sorted['reaction_type'] = None
df_sorted.loc[df_sorted['reactive_to_chain'], 'reaction_type'] = 'competitor_reacts'
df_sorted.loc[df_sorted['reactive_to_competitor'], 'reaction_type'] = 'chain_reacts'



In [29]:
df_sorted['reaction_type'].value_counts()


reaction_type
competitor_reacts    24
chain_reacts         24
Name: count, dtype: int64

In [30]:
df_sorted[df_sorted['reaction_type'].notnull()].groupby('reaction_type')['days_from_prev'].mean()


reaction_type
chain_reacts         3.708333
competitor_reacts    3.166667
Name: days_from_prev, dtype: float64

In [31]:
reactions_summary = df_sorted[df_sorted['reaction_type'].notnull()][[
    'reaction_type', 'competitor', 'chain_campaign', 'start_date',
    'prev_competitor', 'prev_chain_campaign', 'prev_start_date', 'days_from_prev'
]]


In [32]:
competitor_reactions.groupby('prev_chain_campaign').size()


prev_chain_campaign
C1    13
C2    11
dtype: int64

In [33]:
competitor_reactions.groupby('prev_chain_campaign')['days_from_prev'].describe()


,count,mean,std,min,25%,50%,75%,max
prev_chain_campaign,,,,,,,,
C1,13.0,3.230769,0.725011,2.0,3.0,3.0,3.0,5.0
C2,11.0,3.090909,1.136182,0.0,3.0,3.0,4.0,4.0


In [34]:
# Prepare the dataframe for the Gantt chart
df_gantt = df_sorted.copy()
df_gantt['end_date'] = pd.to_datetime(df_gantt['end_date'])  # ensure it's datetime

# Add campaign label for display
df_gantt['campaign_label'] = df_gantt['competitor'] + ' - ' + df_gantt['chain_campaign']

# Map campaign types for clearer display
campaign_type_map = {
    'C1': 'C1 (chain)', 'C2': 'C2 (chain)',
    'A1': 'A1 (competitor)', 'A2': 'A2 (competitor)', 'A3': 'A3 (competitor)'
}
df_gantt['campaign_type'] = df_gantt['chain_campaign'].map(campaign_type_map)

# Optional custom color map
color_discrete_map = {
    'C1 (chain)': 'royalblue',
    'C2 (chain)': 'deepskyblue',
    'A1 (competitor)': 'tomato',
    'A2 (competitor)': 'orangered',
    'A3 (competitor)': 'salmon'
}

# Gantt chart with Plotly
fig = px.timeline(
    df_gantt,
    x_start='start_date',
    x_end='end_date',
    y='competitor',
    color='campaign_type',
    hover_name='campaign_label',
    title='📅 Campaign Timeline: Chain vs CompetitorA',
    color_discrete_map=color_discrete_map
)

fig.update_yaxes(autorange="reversed")  # show earliest dates at the top
fig.update_layout(height=800, xaxis_title="Date", yaxis_title="Competitor")
fig.show()


In [35]:
# Separate data for 'chain' and 'competitorA'
chain_data = df2[df2['competitor'] == 'chain']
competitorA_data = df2[df2['competitor'] == 'competitorA']

# Decompose the time series
# Weekly count of campaigns for 'chain'
chain_weekly_counts = chain_data.resample('W', on='start_date')['chain_campaign'].count().fillna(0)
chain_decomposition = seasonal_decompose(chain_weekly_counts, model='additive', period=12)

# Weekly count of campaigns for 'competitorA'
competitorA_weekly_counts = competitorA_data.resample('W', on='start_date')['chain_campaign'].count().fillna(0)
competitorA_decomposition = seasonal_decompose(competitorA_weekly_counts, model='additive', period=12)

# Create subplots for 'chain'
fig_chain = make_subplots(
    rows=4, cols=1, shared_xaxes=True,
    subplot_titles=["Observed (Chain)", "Trend (Chain)", "Seasonal (Chain)", "Residual (Chain)"],
    vertical_spacing=0.05
)

# Add traces for 'chain'
fig_chain.add_trace(go.Scatter(x=chain_decomposition.observed.index, y=chain_decomposition.observed, name="Observed (Chain)"), row=1, col=1)
fig_chain.add_trace(go.Scatter(x=chain_decomposition.trend.index, y=chain_decomposition.trend, name="Trend (Chain)"), row=2, col=1)
fig_chain.add_trace(go.Scatter(x=chain_decomposition.seasonal.index, y=chain_decomposition.seasonal, name="Seasonal (Chain)"), row=3, col=1)
fig_chain.add_trace(go.Scatter(x=chain_decomposition.resid.index, y=chain_decomposition.resid, name="Residual (Chain)"), row=4, col=1)

# Create subplots for 'competitorA'
fig_competitorA = make_subplots(
    rows=4, cols=1, shared_xaxes=True,
    subplot_titles=["Observed (CompetitorA)", "Trend (CompetitorA)", "Seasonal (CompetitorA)", "Residual (CompetitorA)"],
    vertical_spacing=0.05
)

# Add traces for 'competitorA'
fig_competitorA.add_trace(go.Scatter(x=competitorA_decomposition.observed.index, y=competitorA_decomposition.observed, name="Observed (CompetitorA)"), row=1, col=1)
fig_competitorA.add_trace(go.Scatter(x=competitorA_decomposition.trend.index, y=competitorA_decomposition.trend, name="Trend (CompetitorA)"), row=2, col=1)
fig_competitorA.add_trace(go.Scatter(x=competitorA_decomposition.seasonal.index, y=competitorA_decomposition.seasonal, name="Seasonal (CompetitorA)"), row=3, col=1)
fig_competitorA.add_trace(go.Scatter(x=competitorA_decomposition.resid.index, y=competitorA_decomposition.resid, name="Residual (CompetitorA)"), row=4, col=1)

# Adjust layout and add title for 'chain'
fig_chain.update_layout(
    height=800, width=1000, title_text="Time Series Decomposition (Chain)",
    showlegend=False
)

# Adjust layout and add title for 'competitorA'
fig_competitorA.update_layout(
    height=800, width=1000, title_text="Time Series Decomposition (CompetitorA)",
    showlegend=False
)

# Show the graphs
fig_chain.show()
fig_competitorA.show()



In [36]:
# This script analyzes and compares the campaign activity patterns of two competitors over time.
# It decomposes the weekly frequency of each campaign into trend, seasonal, and residual components to better understand the temporal behavior and strategy behind each campaign.
# The decomposition results are visualized to help identify patterns, shifts, or anomalies in how campaigns are launched and evolve over time for both the 'chain' and 'competitorA'.

chain_data = df2[df2['competitor'] == 'chain']
competitorA_data = df2[df2['competitor'] == 'competitorA']

# Function to decompose time series for each campaign
def decompose_campaign_data(data, campaign_column, period=12):
    # Group by campaign and resample weekly
    campaign_data = data.groupby(campaign_column).resample('W', on='start_date').size().fillna(0)
    
    # Store decomposition results per campaign
    decomposition_results = {}
    for campaign, series in campaign_data.groupby(level=0):
        weekly_counts = series.droplevel(0)
        decomposition = seasonal_decompose(weekly_counts, model='additive', period=period)
        decomposition_results[campaign] = decomposition
        
    return decomposition_results

# Decompose data for 'chain'
chain_decompositions = decompose_campaign_data(chain_data, 'chain_campaign', period=12)

# Decompose data for 'competitorA'
competitorA_decompositions = decompose_campaign_data(competitorA_data, 'chain_campaign', period=12)

# Function to plot decomposition results
def plot_decompositions(decompositions, title):
    # Total number of campaigns
    num_campaigns = len(decompositions)
    
    # Create subplots: 4 rows per campaign (Observed, Trend, Seasonal, Residual)
    fig = make_subplots(
        rows=num_campaigns * 4, cols=1, shared_xaxes=True,
        subplot_titles=[f"Observed ({campaign})" for campaign in decompositions.keys()] + 
                       [f"Trend ({campaign})" for campaign in decompositions.keys()] +
                       [f"Seasonal ({campaign})" for campaign in decompositions.keys()] +
                       [f"Residual ({campaign})" for campaign in decompositions.keys()],
        vertical_spacing=0.05
    )
    
    # Add traces for each component
    row = 1
    for campaign, decomposition in decompositions.items():
        fig.add_trace(go.Scatter(x=decomposition.observed.index, y=decomposition.observed, name=f"Observed ({campaign})"), row=row, col=1)
        row += 1
        fig.add_trace(go.Scatter(x=decomposition.trend.index, y=decomposition.trend, name=f"Trend ({campaign})"), row=row, col=1)
        row += 1
        fig.add_trace(go.Scatter(x=decomposition.seasonal.index, y=decomposition.seasonal, name=f"Seasonal ({campaign})"), row=row, col=1)
        row += 1
        fig.add_trace(go.Scatter(x=decomposition.resid.index, y=decomposition.resid, name=f"Residual ({campaign})"), row=row, col=1)
        row += 1
    
    # Final layout adjustments
    fig.update_layout(
        height=800 * num_campaigns, width=1000, title_text=title,
        showlegend=False
    )
    fig.show()

# Plot decompositions for 'chain'
plot_decompositions(chain_decompositions, "📊 Time Series Decomposition (Chain)")

# Plot decompositions for 'competitorA'
plot_decompositions(competitorA_decompositions, "📊 Time Series Decomposition (CompetitorA)")



In [37]:
# This code identifies reactive campaign pairs based on competitors' campaigns.
# A reactive pair is defined as two consecutive campaigns from different competitors with a time difference of 7 days or less. The code processes the dataset to find these reactive pairs and outputs a dataframe with the details of the competing campaigns.

df_reactive = df2.copy()

df_reactive = df_reactive.sort_values('start_date').reset_index(drop=True)

df_reactive['next_competitor'] = df_reactive['competitor'].shift(-1)
df_reactive['next_start_date'] = df_reactive['start_date'].shift(-1)
df_reactive['next_campaign'] = df_reactive['chain_campaign'].shift(-1) 
df_reactive['days_to_next'] = (df_reactive['next_start_date'] - df_reactive['start_date']).dt.days

df_reactive['is_reactive'] = (
    (df_reactive['competitor'] != df_reactive['next_competitor']) &
    (df_reactive['days_to_next'] <= 7)
)

reactive_pairs = df_reactive[df_reactive['is_reactive']][[
    'competitor', 'start_date', 'chain_campaign',
    'next_competitor', 'next_start_date', 'next_campaign', 'days_to_next'
]]

reactive_pairs.columns = [
    'from_competitor', 'from_start_date', 'from_campaign',
    'to_competitor', 'to_start_date', 'to_campaign', 'reaction_days'
]

reactive_pairs



,from_competitor,from_start_date,from_campaign,to_competitor,to_start_date,to_campaign,reaction_days
5,chain,2023-02-21,C2,competitorA,2023-02-24,A2,3.0
6,competitorA,2023-02-24,A2,chain,2023-02-28,C2,4.0
7,chain,2023-02-28,C2,competitorA,2023-03-03,A1,3.0
9,competitorA,2023-03-10,A1,chain,2023-03-14,C2,4.0
10,chain,2023-03-14,C2,competitorA,2023-03-17,A2,3.0
14,chain,2023-04-11,C1,competitorA,2023-04-14,A2,3.0
16,chain,2023-05-02,C1,competitorA,2023-05-05,A1,3.0
17,competitorA,2023-05-05,A1,chain,2023-05-09,C1,4.0
18,chain,2023-05-09,C1,competitorA,2023-05-12,A2,3.0
20,competitorA,2023-05-18,A2,chain,2023-05-23,C2,5.0


In [38]:
# This code generates a histogram showing the distribution of campaigns by day of the week, 
# grouped by campaign type, with the days of the week ordered correctly.

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df2['start_day'] = pd.Categorical(df2['start_day'], categories=days, ordered=True)

fig = px.histogram(
    df2,
    x='start_day',
    color='chain_campaign',
    barmode='group',         
    title='Distribution of start_day by Campaign Type',
    labels={'start_day': 'Day of the Week', 'count': 'Number of Campaigns'}
)

fig.update_layout(xaxis_title='Day of the Week', yaxis_title='Number of Campaigns')
fig.show()



In [39]:
fig = px.histogram(
    df2,
    x='start_date',               
    facet_col='competitor',
    nbins=22,
    title='Distribution of Campaigns by Month and Competitor',
    labels={'start_month': 'Campaign Start Month'},
    opacity=0.75
)

# Apply 45-degree rotation to X-axis ticks in all subplots
fig.update_xaxes(tickangle=45)

fig.update_layout(
    bargap=0.1,
    xaxis_title='Start Month',
    yaxis_title='Number of Campaigns'
)

fig.show()




In [40]:
# Calculate median campaign duration by campaign type
median_duration = df2.groupby('chain_campaign')['duration_days'].median().reset_index()
median_duration = median_duration.sort_values(by='duration_days', ascending=False)

# Create a bar chart to display the results
fig = px.bar(
    median_duration,
    x='chain_campaign',
    y='duration_days',
    text='duration_days',
    title='Median Campaign Duration by Campaign Type',
    labels={'duration_days': 'Average Duration (days)', 'chain_campaign': 'Campaign'},
    color='chain_campaign'
)

# Update trace formatting
fig.update_traces(texttemplate='%{text:.2f} days', textposition='outside')

# Update layout for better readability
fig.update_layout(
    yaxis_title='Average Duration (days)',
    xaxis_title='Campaign',
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig.show()




In [41]:
# Calculating the mode of duration_days for each chain_campaign
most_frequent_duration = df2.groupby('chain_campaign')['duration_days'].apply(lambda x: x.mode()[0]).reset_index()

# Sorting by most frequent duration
most_frequent_duration = most_frequent_duration.sort_values(by='duration_days', ascending=False)

# Creating the bar chart using Plotly
fig = px.bar(
    most_frequent_duration,
    x='chain_campaign',
    y='duration_days',
    text='duration_days',
    title='Most Frequent Campaign Duration by chain_campaign',
    labels={'duration_days': 'Duration (days)', 'chain_campaign': 'Campaign'},
    color='chain_campaign'
)

# Adding text to bars
fig.update_traces(texttemplate='%{text:.2f} days', textposition='outside')

# Adjusting layout
fig.update_layout(
    yaxis_title='Duration (days)',
    xaxis_title='Campaign',
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig.show()



In [42]:
# This section enriches the sales dataset (df1) by identifying whether each sale occurred during an active campaign period of either the 'chain' or its competitor ('competitorA').
# It matches sales dates against campaign intervals from df2, creating two boolean flags:
# - 'chain_campaign_active': True if the sale happened during a chain campaign
# - 'competitorA_campaign_active': True if the sale happened during a competitorA campaign
# This enables later analysis of campaign overlap, effectiveness, and competitive timing.

df_chain_campaigns = df2[df2['competitor'] == 'chain']

# Merge df1 with 'chain' campaigns based on 'competitor' and date range
def is_campaign_active(row, df_campaigns):
    # Check if the sale date ('time_key') falls within the campaign period
    return ((df_campaigns['start_date'] <= row['time_key']) &
            (df_campaigns['end_date'] >= row['time_key'])).any()

# Apply the function to identify if a sale occurred during a chain campaign
df1['chain_campaign_active'] = df1.apply(
    lambda row: is_campaign_active(row, df_chain_campaigns), axis=1
)

# Filter 'competitorA' campaigns from df2
df_competitorA_campaigns = df2[df2['competitor'] == 'competitorA']

# Function to check if a competitorA campaign was active at a given 'time_key'
def is_competitorA_campaign_active(row, df_campaigns):
    return ((df_campaigns['start_date'] <= row['time_key']) &
            (df_campaigns['end_date'] >= row['time_key'])).any()

# Add a column indicating whether a 'chain' sale occurred during a competitorA campaign
df1['competitorA_campaign_active'] = df1.apply(
    lambda row: is_competitorA_campaign_active(row, df_competitorA_campaigns), axis=1
)





In [43]:
df1

,structure_level_4,structure_level_3,structure_level_2,structure_level_1,sku,time_key,quantity,date,year,month,chain_campaign_active,competitorA_campaign_active
0,3020206,30202,302,3,3111,2023-06-18,18.6840,2023-06-18,2023,6,True,True
1,3020608,30206,302,3,3278,2024-07-31,396.1008,2024-07-31,2024,7,True,False
2,3020809,30208,302,3,3603,2023-08-07,6.2280,2023-08-07,2023,8,True,False
3,3020608,30206,302,3,4604,2023-01-31,27.4032,2023-01-31,2023,1,True,False
4,3040808,30408,304,3,3041,2023-09-06,6.2280,2023-09-06,2023,9,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1864589,3020809,30208,302,3,1694,2023-05-31,100.8936,2023-05-31,2023,5,True,True
1864590,2020604,20206,202,2,3314,2023-03-08,66.0168,2023-03-08,2023,3,False,False
1864591,3040601,30406,304,3,2318,2023-03-07,47.3328,2023-03-07,2023,3,False,False
1864592,3020401,30204,302,3,4443,2023-02-04,382.3992,2023-02-04,2023,2,True,False


## Hypothesis Tests

# Analyze whether sales performance differs when the competitorA is inactive, comparing periods when the chain is running a campaign versus when it is not.

# This isolates the effect of the chain's own campaigns in the absence of competitor interference.


In [42]:
# Filtrar apenas quando o concorrente está inativo
df_comp_inactive = df1[df1['competitorA_campaign_active'] == False]

# Criar dois grupos: cadeia com campanha ativa e inativa
df_chain_active = df_comp_inactive[df_comp_inactive['chain_campaign_active'] == True]
df_chain_inactive = df_comp_inactive[df_comp_inactive['chain_campaign_active'] == False]

# Lista para armazenar os resultados
results = []

# Para cada 'structure_level_4', realizar o teste de hipótese
for level in df1['structure_level_4'].unique():
    # Filtrar os dados para o nível atual
    active_level = df_chain_active[df_chain_active['structure_level_4'] == level]['quantity']
    inactive_level = df_chain_inactive[df_chain_inactive['structure_level_4'] == level]['quantity']
    
    # Verifica se há pelo menos 2 valores em ambas as amostras
    if len(active_level) > 1 and len(inactive_level) > 1:
        # Teste de Mann-Whitney U (não paramétrico)
        u_stat, p_value = stats.mannwhitneyu(active_level, inactive_level, alternative='two-sided')
        test_type = 'Mann-Whitney U test'
        
        # Armazena o resultado
        results.append({
            'structure_level_4': level,
            'test_type': test_type,
            'p_value': p_value
        })

# Converter resultados para DataFrame
results_df = pd.DataFrame(results)

# Interpretação com nível de significância
alpha = 0.05
results_df['significance'] = results_df['p_value'].apply(lambda p: 'Reject H0' if p < alpha else 'Do not reject H0')

# Contagem dos resultados
rejection_count = results_df['significance'].value_counts().reset_index()
rejection_count.columns = ['Significance', 'Quantity']

# Cálculo da percentagem
total_tests = results_df.shape[0]
rejection_count['Percentage'] = (rejection_count['Quantity'] / total_tests * 100).round(2)

# Exibir percentagens
print("\nPercentagem de resultados:")
print(rejection_count)

# Plot com Plotly
fig = px.bar(rejection_count, 
             x='Significance', 
             y='Quantity', 
             color='Significance', 
             title='Number of Rejections and Non-Rejections of the Null Hypothesis',
             labels={'Significance': 'Hypothesis Test Result (Reject or Do not Reject H0)', 
                     'Quantity': 'Number of Structure Levels 4'},
             color_discrete_sequence=px.colors.qualitative.Set2)

# Layout ajustado
fig.update_layout(
    xaxis_title='Hypothesis Test Result',
    yaxis_title='Number of Structure Levels',
    showlegend=False,
    width=600,
    height=400
)

fig.show()





Percentagem de resultados:
       Significance  Quantity  Percentage
0  Do not reject H0       203       55.46
1         Reject H0       163       44.54


In [43]:
# Calcule a mediana das vendas por nível de 'structure_level_4' para campanhas do competitor ativas e inativas
median_active = df_chain_active.groupby('structure_level_4')['quantity'].median()
median_inactive = df_chain_inactive.groupby('structure_level_4')['quantity'].median()

# Compare as medianas
comparison_df = pd.DataFrame({
    'median_active': median_active,
    'median_inactive': median_inactive
})

# Adicionar uma coluna para indicar qual grupo tem a maior mediana
comparison_df['higher_sales'] = comparison_df.apply(
    lambda row: 'Active' if row['median_active'] > row['median_inactive'] else 'Inactive',
    axis=1
)

print(comparison_df)

# Contar a quantidade de "Active" e "Inactive" na coluna 'higher_sales'
higher_sales_count = comparison_df['higher_sales'].value_counts()

higher_sales_df = higher_sales_count.reset_index()
higher_sales_df.columns = ['Competitor Campaign Status', 'Count']

# Adicionar coluna de percentagem
total = higher_sales_df['Count'].sum()
higher_sales_df['Percentage'] = (higher_sales_df['Count'] / total * 100).round(2)

# Create the barplot com texto incluindo percentagem
fig = px.bar(
    higher_sales_df,
    x='Competitor Campaign Status',
    y='Count',
    color='Competitor Campaign Status',
    title="Comparison of Median Sales by Competitor's Campaign Status",
    text=higher_sales_df.apply(lambda row: f"{row['Count']} ({row['Percentage']}%)", axis=1)
)

fig.update_layout(showlegend=False)
fig.show()

                   median_active  median_inactive higher_sales
structure_level_4                                             
1010101               979.664400      1916.355600     Inactive
1010106                12.748716         9.285948       Active
1010107               621.517032       579.490488       Active
1010201              2557.839600      2668.075200     Inactive
1010206               554.292000       467.100000       Active
...                          ...              ...          ...
3051108                11.210400        11.210400     Inactive
3051109                 8.719200         8.096400       Active
3051113                46.087200        39.859200       Active
3051114                 7.473600         6.228000       Active
3051115                14.947200        13.701600       Active

[366 rows x 3 columns]


The null hypothesis (H0) represents the assumption that there is no significant difference between the two groups being tested.
In this case, it implies that there is no significant difference in the number of products sold when the chain's campaign is active versus inactive, assuming the competitor's campaign is not active.

If we reject H0, it means we have sufficient evidence to conclude that a significant difference exists. If we fail to reject H0, it means there is not enough evidence to conclude that a difference exists.

Although the difference was statistically significant in fewer than 50% of the cases, the higher median trend suggests that the campaign still provides a tangible benefit.

The results indicate that the chain's campaign appears to be effective in increasing sales when the competitor is inactive, as reflected by the higher median sales per structure_level_4.

# When competitorA has an active campaign, should the chain also run a campaign in response?
# This analysis compares sales performance during competitorA campaign periods,
# distinguishing between times when the chain is also active versus inactive.


In [46]:
# Objective:
# This analysis investigates whether the chain's sales performance differs
# depending on whether it responds with an active campaign while competitorA is also running a campaign.
# For each 'structure_level_4', a Mann-Whitney U test is applied to compare the sales distribution
# between periods when the chain is active vs inactive during competitorA's campaign.
# The goal is to assess whether reactive campaigning leads to statistically significant changes in sales.

# Filter data where competitorA campaign is active
df_comp_active = df1[df1['competitorA_campaign_active'] == True]

# Create two groups: chain with active campaign and inactive campaign
df_comp_active_chain_active = df_comp_active[df_comp_active['chain_campaign_active'] == True]
df_comp_active_chain_inactive = df_comp_active[df_comp_active['chain_campaign_active'] == False]

# List to store test results
results = []

# Perform hypothesis test for each 'structure_level_4'
for level in df1['structure_level_4'].unique():
    active_level = df_comp_active_chain_active[df_comp_active_chain_active['structure_level_4'] == level]['quantity']
    inactive_level = df_comp_active_chain_inactive[df_comp_active_chain_inactive['structure_level_4'] == level]['quantity']

    if len(active_level) > 1 and len(inactive_level) > 1:
        # Teste de Mann-Whitney U (não paramétrico)
        u_stat, p_value = stats.mannwhitneyu(active_level, inactive_level, alternative='two-sided')
        test_type = 'Mann-Whitney U test'
        
        results.append({
            'structure_level_4': level,
            'test_type': test_type,
            'p_value': p_value
        })


# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Interpretation with significance level
alpha = 0.05
results_df['significance'] = results_df['p_value'].apply(lambda p: 'Reject H0' if p < alpha else 'Do not reject H0')

# Count of results
significance_counts = results_df['significance'].value_counts().reset_index()
significance_counts.columns = ['Significance', 'Count']

# Calculate percentage
total_tests = results_df.shape[0]
significance_counts['Percentage'] = (significance_counts['Count'] / total_tests * 100).round(2)

# Display percentages
print("\nPercentage of test results:")
print(significance_counts)

# Plot with Plotly
fig = px.bar(significance_counts, 
             x='Significance', 
             y='Count', 
             color='Significance', 
             title='Number of Rejections and Non-Rejections of the Null Hypothesis',
             labels={'Significance': 'Hypothesis Test Result (Reject or Do not Reject H0)', 
                     'Count': 'Number of Structure Level 4'},
             color_discrete_sequence=px.colors.qualitative.Set2)

fig.update_layout(
    xaxis_title='Hypothesis Test Result',
    yaxis_title='Number of Structure Levels',
    showlegend=False,
    width=600,
    height=400
)

fig.show()



Percentage of test results:
       Significance  Count  Percentage
0  Do not reject H0    209        57.1
1         Reject H0    157        42.9


In [47]:
# Calculate median sales per 'structure_level_4' for competitor campaigns active and inactive
median_active = df_comp_active_chain_active.groupby('structure_level_4')['quantity'].median()
median_inactive = df_comp_active_chain_inactive.groupby('structure_level_4')['quantity'].median()

# Compare the medians
comparison_df = pd.DataFrame({
    'median_active': median_active,
    'median_inactive': median_inactive
})

# Add a column to indicate which group has the higher median sales
comparison_df['higher_sales'] = comparison_df.apply(
    lambda row: 'Active' if row['median_active'] > row['median_inactive'] else 'Inactive',
    axis=1
)

print(comparison_df)

# Count the number of 'Active' and 'Inactive' in the 'higher_sales' column
higher_sales_count = comparison_df['higher_sales'].value_counts()

higher_sales_df = higher_sales_count.reset_index()
higher_sales_df.columns = ['Competitor Campaign Status', 'Count']

# Add a percentage column
total = higher_sales_df['Count'].sum()
higher_sales_df['Percentage'] = (higher_sales_df['Count'] / total * 100).round(2)

# Create the bar plot with text showing count and percentage
fig = px.bar(
    higher_sales_df,
    x='Competitor Campaign Status',
    y='Count',
    color='Competitor Campaign Status',
    title="Comparison of Median Sales by Competitor's Campaign Status",
    text=higher_sales_df.apply(lambda row: f"{row['Count']} ({row['Percentage']}%)", axis=1)
)

fig.update_layout(showlegend=False)
fig.show()




                   median_active  median_inactive higher_sales
structure_level_4                                             
1010101              2564.690400      2879.827200     Inactive
1010106                18.877068        14.648256       Active
1010107               354.435480       383.719536     Inactive
1010201              4387.538808      3555.203976       Active
1010206               805.280400       772.272000       Active
...                          ...              ...          ...
3051108                 9.964800         9.964800     Inactive
3051109                 8.719200         8.719200     Inactive
3051113                47.332800        42.350400       Active
3051114                 6.228000         6.228000     Inactive
3051115                14.947200        13.701600       Active

[366 rows x 3 columns]


Although statistical significance is not confirmed in most segments, there is a clear pattern in which the chain's campaign tends to increase sales — with a higher median in nearly 80% of cases. This suggests a real and consistent effect, even if not always statistically detectable, which supports the consideration of reactive campaigns in competitive contexts.

As a next step, i recommend conducting an ROI (Return on Investment) analysis of these reactive campaigns. Evaluating whether the increase in sales is sufficient to cover the campaign costs will enable a more informed decision regarding their continuation or expansion. This evaluation should take into account the campaign costs, the contribution margin of the products sold, and potential side effects such as cannibalization or margin erosion.

# Chain Campaign inactive - When chain's campaign is inactive, do sales differ depending on whether the competitor's campaign is active or inactive?


In [48]:
# Filter only when the competitor is inactive
df_chain_inactive = df1[df1['chain_campaign_active'] == False]

# Create two groups: chain with active and inactive campaign
df_comp_active = df_chain_inactive[df_chain_inactive['competitorA_campaign_active'] == True]
df_comp_inactive = df_chain_inactive[df_chain_inactive['competitorA_campaign_active'] == False]

# List to store results
results = []

# For each 'structure_level_4', perform hypothesis testing
for level in df1['structure_level_4'].unique():
    # Filter data for the current level
    active_level = df_comp_active[df_comp_active['structure_level_4'] == level]['quantity']
    inactive_level = df_comp_inactive[df_comp_inactive['structure_level_4'] == level]['quantity']
    
    # Check if there are at least 2 values in both samples
    if len(active_level) > 1 and len(inactive_level) > 1:
        # Mann-Whitney U test (non-parametric)
        u_stat, p_value = stats.mannwhitneyu(active_level, inactive_level, alternative='two-sided')
        test_type = 'Mann-Whitney U test'
        
        # Store the result
        results.append({
            'structure_level_4': level,
            'test_type': test_type,
            'p_value': p_value
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Interpretation with significance level
alpha = 0.05
results_df['significance'] = results_df['p_value'].apply(lambda p: 'Reject H0' if p < alpha else 'Do not reject H0')

# Count results
rejection_count = results_df['significance'].value_counts().reset_index()
rejection_count.columns = ['Significance', 'Quantity']

# Calculate percentage
total_tests = results_df.shape[0]
rejection_count['Percentage'] = (rejection_count['Quantity'] / total_tests * 100).round(2)

# Display percentages
print("\nPercentage of results:")
print(rejection_count)

# Plot with Plotly
fig = px.bar(rejection_count, 
             x='Significance', 
             y='Quantity', 
             color='Significance', 
             title='Number of Rejections and Non-Rejections of the Null Hypothesis',
             labels={'Significance': 'Hypothesis Test Result (Reject or Do not Reject H0)', 
                     'Quantity': 'Number of Structure Levels 4'},
             color_discrete_sequence=px.colors.qualitative.Set2)

# Adjusted layout
fig.update_layout(
    xaxis_title='Hypothesis Test Result',
    yaxis_title='Number of Structure Levels',
    showlegend=False,
    width=600,
    height=400
)

fig.show()



Percentage of results:
       Significance  Quantity  Percentage
0         Reject H0       211       57.65
1  Do not reject H0       155       42.35


In [49]:
# Calculate the median sales by 'structure_level_4' for active and inactive competitor campaigns
median_active = df_comp_active.groupby('structure_level_4')['quantity'].median()
median_inactive = df_comp_inactive.groupby('structure_level_4')['quantity'].median()

# Compare the medians
comparison_df = pd.DataFrame({
    'median_active': median_active,
    'median_inactive': median_inactive
})

# Add a column to indicate which group has the higher median
comparison_df['higher_sales'] = comparison_df.apply(
    lambda row: 'Active' if row['median_active'] > row['median_inactive'] else 'Inactive',
    axis=1
)

print(comparison_df)

# Count the number of "Active" and "Inactive" in the 'higher_sales' column
higher_sales_count = comparison_df['higher_sales'].value_counts()

higher_sales_df = higher_sales_count.reset_index()
higher_sales_df.columns = ['Competitor Campaign Status', 'Count']

# Add percentage column
total = higher_sales_df['Count'].sum()
higher_sales_df['Percentage'] = (higher_sales_df['Count'] / total * 100).round(2)

# Create the barplot with text including percentage
fig = px.bar(
    higher_sales_df,
    x='Competitor Campaign Status',
    y='Count',
    color='Competitor Campaign Status',
    title="Comparison of Median Sales by Competitor's Campaign Status",
    text=higher_sales_df.apply(lambda row: f"{row['Count']} ({row['Percentage']}%)", axis=1)
)

fig.update_layout(showlegend=False)
fig.show()


                   median_active  median_inactive higher_sales
structure_level_4                                             
1010101              2879.827200      1916.355600       Active
1010106                14.648256         9.285948       Active
1010107               383.719536       579.490488     Inactive
1010201              3555.203976      2668.075200       Active
1010206               772.272000       467.100000       Active
...                          ...              ...          ...
3051108                 9.964800        11.210400     Inactive
3051109                 8.719200         8.096400       Active
3051113                42.350400        39.859200       Active
3051114                 6.228000         6.228000     Inactive
3051115                13.701600        13.701600     Inactive

[366 rows x 3 columns]


Even when the chain's campaigns are inactive, the results show that competitor activity significantly influences sales in more than half of the analyzed segments (58% with H0 rejected). Even more striking is that in 82% of cases, sales were higher when the competitor had an active campaign, suggesting a possible indirect positive effect or a spillover in demand.
This highlights the importance of not only monitoring your own promotional actions but also closely tracking competitor strategies, even when your chain is not actively promoting.


# Chain Campaign active - When chain has an active campaign, does the competitor being active as well affect sales?


In [50]:
# Filter only when the chain campaign is active
df_chain_active = df1[df1['chain_campaign_active'] == True]

# Create two groups: chain campaign active and inactive competitor campaigns
df_chain_active_comp_active = df_chain_active[df_chain_active['competitorA_campaign_active'] == True]
df_chain_active_comp_inactive = df_chain_active[df_chain_active['competitorA_campaign_active'] == False]

# List to store the results
results = []

# For each 'structure_level_4', perform hypothesis testing
for level in df1['structure_level_4'].unique():
    # Filter data for the current level
    active_level = df_chain_active_comp_active[df_chain_active_comp_active['structure_level_4'] == level]['quantity']
    inactive_level = df_chain_active_comp_inactive[df_chain_active_comp_inactive['structure_level_4'] == level]['quantity']
    
    # Check if both samples have at least 2 values
    if len(active_level) > 1 and len(inactive_level) > 1:
        # Mann-Whitney U test (non-parametric)
        u_stat, p_value = stats.mannwhitneyu(active_level, inactive_level, alternative='two-sided')
        test_type = 'Mann-Whitney U test'
        
        # Store the result
        results.append({
            'structure_level_4': level,
            'test_type': test_type,
            'p_value': p_value
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Interpretation with significance level
alpha = 0.05
results_df['significance'] = results_df['p_value'].apply(lambda p: 'Reject H0' if p < alpha else 'Do not reject H0')

# Count the results
rejection_count = results_df['significance'].value_counts().reset_index()
rejection_count.columns = ['Significance', 'Quantity']

# Calculate the percentage
total_tests = results_df.shape[0]
rejection_count['Percentage'] = (rejection_count['Quantity'] / total_tests * 100).round(2)

# Print percentages
print("\nPercentage of results:")
print(rejection_count)

# Plot with Plotly
fig = px.bar(rejection_count, 
             x='Significance', 
             y='Quantity', 
             color='Significance', 
             title='Number of Rejections and Non-Rejections of the Null Hypothesis',
             labels={'Significance': 'Hypothesis Test Result (Reject or Do not Reject H0)', 
                     'Quantity': 'Number of Structure Levels 4'},
             color_discrete_sequence=px.colors.qualitative.Set2)

# Adjusted layout
fig.update_layout(
    xaxis_title='Hypothesis Test Result',
    yaxis_title='Number of Structure Levels',
    showlegend=False,
    width=600,
    height=400
)

fig.show()



Percentage of results:
       Significance  Quantity  Percentage
0         Reject H0       238       65.03
1  Do not reject H0       128       34.97


In [49]:
# Calcule a mediana das vendas por nível de 'structure_level_4' para campanhas do competitor ativas e inativas
median_active = df_chain_active_comp_active.groupby('structure_level_4')['quantity'].median()
median_inactive = df_chain_active_comp_inactive.groupby('structure_level_4')['quantity'].median()

# Compare as medianas
comparison_df = pd.DataFrame({
    'median_active': median_active,
    'median_inactive': median_inactive
})

# Adicionar uma coluna para indicar qual grupo tem a maior mediana
comparison_df['higher_sales'] = comparison_df.apply(
    lambda row: 'Active' if row['median_active'] > row['median_inactive'] else 'Inactive',
    axis=1
)

print(comparison_df)

# Contar a quantidade de "Active" e "Inactive" na coluna 'higher_sales'
higher_sales_count = comparison_df['higher_sales'].value_counts()

higher_sales_df = higher_sales_count.reset_index()
higher_sales_df.columns = ['Competitor Campaign Status', 'Count']

# Adicionar coluna de percentagem
total = higher_sales_df['Count'].sum()
higher_sales_df['Percentage'] = (higher_sales_df['Count'] / total * 100).round(2)

# Create the barplot com texto incluindo percentagem
fig = px.bar(
    higher_sales_df,
    x='Competitor Campaign Status',
    y='Count',
    color='Competitor Campaign Status',
    title="Comparison of Median Sales by Competitor's Campaign Status",
    text=higher_sales_df.apply(lambda row: f"{row['Count']} ({row['Percentage']}%)", axis=1)
)

fig.update_layout(showlegend=False)
fig.show()

                   median_active  median_inactive higher_sales
structure_level_4                                             
1010101              2564.690400       979.664400       Active
1010106                18.877068        12.748716       Active
1010107               354.435480       621.517032     Inactive
1010201              4387.538808      2557.839600       Active
1010206               805.280400       554.292000       Active
...                          ...              ...          ...
3051108                 9.964800        11.210400     Inactive
3051109                 8.719200         8.719200     Inactive
3051113                47.332800        46.087200       Active
3051114                 6.228000         7.473600     Inactive
3051115                14.947200        14.947200     Inactive

[366 rows x 3 columns]


When the chain has an active campaign, competitor activity still plays a relevant role: in 65% of segments, there was a statistically significant difference in sales, and in 82% of cases, the median sales were higher when the competitor was also active.
This suggests that simultaneous campaigns do not necessarily harm the chain's performance — on the contrary, they may amplify category demand and have a positive effect on sales.

In [ ]:
# This linear regression model evaluates the impact of chain campaign activity, competitor A campaign activity, and their interaction effect on sales quantity.

model = smf.ols('quantity ~ chain_campaign_active + competitorA_campaign_active + chain_campaign_active:competitorA_campaign_active', data=df1).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:               quantity   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     258.4
Date:                Thu, 15 May 2025   Prob (F-statistic):          1.16e-167
Time:                        12:40:45   Log-Likelihood:            -1.7400e+07
No. Observations:             1864302   AIC:                         3.480e+07
Df Residuals:                 1864298   BIC:                         3.480e+07
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

The regression model shows that both the chain's own campaign and the competitor’s campaign are associated with statistically significant increases in sales. Notably, competitor activity alone contributes more to sales than the chain’s own campaign, suggesting a potential spillover effect. Moreover, when both campaigns are active simultaneously, there is an additional positive interaction, indicating that such scenarios may further amplify demand. Although the model explains only a small portion of the overall variance in sales (low R²), the direction and significance of the coefficients provide valuable strategic insight.

Results' Analysis

This study evaluates the effectiveness of the chain's sales campaigns compared to the competitor's campaign activity, aiming to understand the impact of promotional actions in different scenarios. The research uses the null hypothesis (H0) to assess whether there is a significant difference in sales when the chain's campaign is active versus inactive, assuming the competitor's campaign is not active. From the observed results, several important insights can be drawn to guide the company's strategic marketing decisions.

1. Null Hypothesis (H0) and Statistical Meaning:
The null hypothesis (H0) assumes that there is no significant difference in the number of products sold when the chain's campaign is active compared to when it is inactive, assuming the competitor’s campaign is not active. If the analytical data suggests that H0 should be rejected, it implies that there is a statistically significant difference in sales between the two scenarios. If H0 is not rejected, it means there is insufficient evidence to claim that a significant difference exists between the active and inactive campaign states.

2. Evidence of Effectiveness of the Chain's Campaigns:
Although the statistical difference was significant in fewer than 50% of the cases analyzed, a consistent trend is observed in which the chain's campaign tends to result in a higher median of sales, suggesting that there is a tangible benefit, even if not always clearly detected statistically. When the chain's campaign is active and the competitor's campaign is inactive, sales are typically higher. This trend is particularly strong, as the median sales per structure_level_4 (structural level of analysis) indicate that the chain's campaign is, in most cases, generating an increase in sales.

Although statistical significance was not confirmed across all analyzed segments, a clear pattern emerges: in about 80% of the cases, the chain's campaign shows a higher median of sales when active, indicating a real and consistent effect. This suggests that the chain's campaign has a positive impact on sales, even if it is not always statistically detectable. This reinforces the hypothesis that reactive campaigns in a competitive context can indeed have a positive effect on sales, even if that impact is not always statistically significant.

3. Recommendation for ROI (Return on Investment) Analysis:
Since the chain’s campaign seems to have a positive effect on sales, although not always statistically significant, it is recommended to conduct an ROI (Return on Investment) analysis to assess whether the increase in sales justifies the campaign’s costs. A well-structured ROI analysis will provide a more precise evaluation of the feasibility and continuity of the campaigns.

The ROI analysis should take into account the following factors:

Campaign cost: Determine the total cost associated with running the campaign, including all marketing and resource expenses.

Contribution margin: Analyze the impact of additional sales on the contribution margin of the products sold.

Side effects: Assess potential unwanted effects, such as sales cannibalization (when internal campaigns steal sales from other products or channels) or margin erosion (a decrease in profit margin due to discounts or other promotions).

Carrying out this analysis will allow for more informed decisions regarding the continuation or expansion of the reactive campaigns.

4. Impact of Competitor Campaigns on Sales:
Even when the chain's campaign is inactive, the results indicate that the competitor's activity has a significant impact on sales in more than 50% of the analyzed segments (58% with H0 rejected). In a striking 82% of cases, sales were higher when the competitor's campaign was active, suggesting a possible positive indirect effect or "spillover" of demand due to the competitor’s activity. This phenomenon highlights the importance of not only monitoring one's own promotional activities but also tracking competitor strategies closely. This is especially relevant in contexts where the chain is not running active promotions, as the competitor's actions can directly influence sales results.

5. Effect of Simultaneous Campaigns:
Another important point that emerges from the analysis is the impact of simultaneous campaigns from both the chain and the competitor. When the chain has an active campaign, the competitor’s activity still plays a relevant role. In 65% of the analyzed segments, there was a statistically significant difference in sales. Furthermore, in 83% of the cases, the median sales were higher when both campaigns were active at the same time. These data suggest that simultaneous campaigns do not necessarily harm the chain’s performance; on the contrary, they can amplify market demand and have a positive effect on sales, increasing overall demand for the category. This positive interaction between campaigns is a valuable insight for formulating marketing strategies in competitive environments.

6. Regression Model and Campaign Impact Analysis:
The regression model used in the study revealed that both the chain's campaign and the competitor's campaign are associated with statistically significant increases in sales. Notably, the competitor’s activity seems to contribute more to the increase in sales than the chain's own campaign. This indicates a potential "spillover" effect, where the competitor's promotional actions influence the chain’s sales. Additionally, when both campaigns are active, there is an additional positive interaction that further amplifies demand, increasing sales.

Although the regression model explains only a small portion of the sales variance (with a low R² value), the results are still valuable. The direction and significance of the coefficients suggest that promotional campaigns have a positive impact on sales, and that the interaction between the chain's and competitor's campaigns can be an important strategic factor to consider.

Conclusions:
Positive impact of the chain's campaigns: The chain's campaigns have a positive impact on sales, especially when the competitor’s campaign is inactive. The median sales are consistently higher when the chain’s campaign is active, even though statistical significance is not always observed.

Importance of competitor behavior: The competitor’s activity has a considerable impact on the chain's sales, with a significant "spillover" effect observed in more than 80% of cases.

Simultaneous campaigns: Simultaneous campaigns from the chain and the competitor tend to increase market demand, with a positive effect on sales. This suggests that there is no direct negative competition between the campaigns but rather an amplification of demand.

Continuous monitoring: To make effective strategic decisions, it is crucial to monitor not only one's own campaigns but also the actions of competitors, as their strategies can have a significant impact, even when the chain is not actively promoting.

ROI analysis recommended: Conducting a detailed ROI analysis will help determine the financial viability of the campaigns, considering both the increase in sales and the associated costs, as well as potential side effects.

# Product prices leaflets

In [52]:
# Product prices leaflets
df3 = pd.read_csv("train/product_prices_leaflets.csv")
df3.head(20)

,sku,time_key,competitor,pvp_was,discount,flag_promo,leaflet
0,2056,20240312,chain,21.70,0.0000,0,NaN
1,4435,20230621,chain,18.92,0.2754,1,weekly
2,1951,20230303,competitorA,60.58,0.3524,1,NaN
3,2135,20240605,chain,55.37,0.2509,1,weekly
4,3949,20230529,chain,51.90,0.0000,0,NaN
5,1683,20231126,competitorA,62.14,0.1117,1,NaN
6,3302,20240427,chain,39.75,0.0000,0,NaN
7,2937,20230405,chain,17.18,0.0000,0,NaN
8,2309,20231220,competitorA,77.93,0.0000,0,NaN
9,1598,20240815,chain,32.98,0.0000,0,NaN


In [53]:
df3.shape


(3112140, 7)

In [54]:
df3.describe()

,sku,time_key,pvp_was,discount,flag_promo
count,3.112140e+06,3.112140e+06,3.112140e+06,3.112140e+06,3.112140e+06
mean,3.171605e+03,2.023514e+07,5.397732e+01,6.569260e-02,2.449617e-01
std,9.778981e+02,4.935283e+03,6.324775e+01,1.235808e-01,4.300645e-01
min,1.128000e+03,2.023010e+07,3.990000e+00,-2.030700e+00,0.000000e+00
25%,2.351000e+03,2.023062e+07,2.673000e+01,0.000000e+00,0.000000e+00
50%,3.242000e+03,2.023113e+07,3.975000e+01,0.000000e+00,0.000000e+00
75%,4.025000e+03,2.024051e+07,5.710000e+01,0.000000e+00,0.000000e+00
max,4.735000e+03,2.024103e+07,1.084640e+03,9.255000e-01,1.000000e+00


In [55]:
df3 = df3[df3['discount'] >= 0]



In [56]:
df3['pvp_final'] = (df3['pvp_was'] * (1 - df3['discount'])).round(2)

df3['time_key'] = pd.to_datetime(df3['time_key'], format='%Y%m%d')
df3['year'] = pd.to_datetime(df3['time_key']).dt.year
df3['month'] = pd.to_datetime(df3['time_key']).dt.month
df3['day_of_week'] = pd.to_datetime(df3['time_key']).dt.dayofweek  # 0=segunda
df3['week'] = pd.to_datetime(df3['time_key']).dt.isocalendar().week
df3['is_weekend'] = df3['day_of_week'].isin([5, 6]).astype(int)



In [58]:
df_inconsistents = df3[(df3['flag_promo'] == 0) & (df3['discount'] != 0)]
df_inconsistents.head()

,sku,time_key,competitor,pvp_was,discount,flag_promo,leaflet,pvp_final,year,month,day_of_week,week,is_weekend
226,4063,2024-08-23,competitorB,20.65,0.0416,0,NaN,19.79,2024,8,4,34,0
552,2624,2023-04-21,competitorB,35.93,0.0434,0,NaN,34.37,2023,4,4,16,0
812,2619,2023-05-22,competitorB,24.13,0.0290,0,NaN,23.43,2023,5,0,21,0
1042,4569,2024-02-06,competitorB,15.10,0.0457,0,NaN,14.41,2024,2,1,6,0
1325,1927,2023-05-16,competitorA,70.12,0.0222,0,weekly,68.56,2023,5,1,20,0


In [59]:
df_inconsistents.shape

(7909, 13)

In [60]:
df3.loc[(df3['flag_promo'] == 0) & (df3['discount'] != 0), 'flag_promo'] = 1

In [61]:
def add_seasonal_promo_features(df3):
    df = df3.copy()
    
    # Garantir que time_key está em datetime
    df['time_key'] = pd.to_datetime(df['time_key'])
    
    # Features sazonais
    df['month'] = df['time_key'].dt.month
    df['day_of_week'] = df['time_key'].dt.dayofweek
    
    df['is_christmas_season'] = df['month'].isin([12]).astype(int)
    df['is_new_year'] = df['time_key'].between('2023-12-26', '2024-01-05').astype(int)
    df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
    df['is_back_to_school'] = (df['month'] == 9).astype(int)
    df['is_black_friday'] = df['time_key'].apply(
        lambda x: x.month == 11 and x.weekday() == 4 and 23 <= x.day <= 29
    ).astype(int)

    # Promoções em épocas específicas
    df['promo_in_christmas'] = ((df['is_christmas_season'] == 1) & (df['flag_promo'] == 1)).astype(int)
    df['promo_in_summer'] = ((df['is_summer'] == 1) & (df['flag_promo'] == 1)).astype(int)
    df['promo_in_back_to_school'] = ((df['is_back_to_school'] == 1) & (df['flag_promo'] == 1)).astype(int)
    df['promo_in_black_friday'] = ((df['is_black_friday'] == 1) & (df['flag_promo'] == 1)).astype(int)

    # Flag geral de promoção sazonal
    df['is_seasonal_promo'] = (
        (df['flag_promo'] == 1) &
        (
            (df['is_christmas_season'] == 1) |
            (df['is_summer'] == 1) |
            (df['is_back_to_school'] == 1) |
            (df['is_black_friday'] == 1)
        )
    ).astype(int)
    
    return df

df3 = add_seasonal_promo_features(df3)


In [62]:
df3.head()

,sku,time_key,competitor,pvp_was,discount,flag_promo,leaflet,pvp_final,year,month,...,is_christmas_season,is_new_year,is_summer,is_back_to_school,is_black_friday,promo_in_christmas,promo_in_summer,promo_in_back_to_school,promo_in_black_friday,is_seasonal_promo
0,2056,2024-03-12,chain,21.70,0.0000,0,NaN,21.70,2024,3,...,0,0,0,0,0,0,0,0,0,0
1,4435,2023-06-21,chain,18.92,0.2754,1,weekly,13.71,2023,6,...,0,0,1,0,0,0,1,0,0,1
2,1951,2023-03-03,competitorA,60.58,0.3524,1,NaN,39.23,2023,3,...,0,0,0,0,0,0,0,0,0,0
3,2135,2024-06-05,chain,55.37,0.2509,1,weekly,41.48,2024,6,...,0,0,1,0,0,0,1,0,0,1
4,3949,2023-05-29,chain,51.90,0.0000,0,NaN,51.90,2023,5,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# Objective: Calculate the price difference between competitors 'competitorA' and 'competitorB' relative to the 'chain' for each (time_key, sku) combination.
# This allows analysis of how competitors' prices compare to the main chain's prices over time.

# First, get the final price (pvp_final) of the 'chain' competitor for each (time_key, sku)
chain_prices = df3[df3['competitor'] == 'chain'][['time_key', 'sku', 'pvp_final']].rename(columns={'pvp_final': 'pvp_final_chain'})

# Merge the chain prices back into the original dataset
df3 = df3.merge(chain_prices, on=['time_key', 'sku'], how='left')

# Create the column diff_A: difference between competitorA's price and chain's price, for competitorA rows only
df3['diff_A'] = df3.apply(
    lambda row: row['pvp_final'] - row['pvp_final_chain'] if row['competitor'] == 'competitorA' else None,
    axis=1
)

# Create the column diff_B: difference between competitorB's price and chain's price, for competitorB rows only
df3['diff_B'] = df3.apply(
    lambda row: row['pvp_final'] - row['pvp_final_chain'] if row['competitor'] == 'competitorB' else None,
    axis=1
)

# Drop the auxiliary 'pvp_final_chain' column
df3 = df3.drop(columns=['pvp_final_chain'])



In [64]:
df3

,sku,time_key,competitor,pvp_was,discount,flag_promo,leaflet,pvp_final,year,month,...,is_summer,is_back_to_school,is_black_friday,promo_in_christmas,promo_in_summer,promo_in_back_to_school,promo_in_black_friday,is_seasonal_promo,diff_A,diff_B
0,2056,2024-03-12,chain,21.70,0.0000,0,NaN,21.70,2024,3,...,0,0,0,0,0,0,0,0,NaN,NaN
1,4435,2023-06-21,chain,18.92,0.2754,1,weekly,13.71,2023,6,...,1,0,0,0,1,0,0,1,NaN,NaN
2,1951,2023-03-03,competitorA,60.58,0.3524,1,NaN,39.23,2023,3,...,0,0,0,0,0,0,0,0,-21.35,NaN
3,2135,2024-06-05,chain,55.37,0.2509,1,weekly,41.48,2024,6,...,1,0,0,0,1,0,0,1,NaN,NaN
4,3949,2023-05-29,chain,51.90,0.0000,0,NaN,51.90,2023,5,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288059,4726,2024-06-08,chain,70.99,0.5158,1,weekly,34.37,2024,6,...,1,0,0,0,1,0,0,1,NaN,NaN
3288060,4692,2023-06-26,competitorA,18.92,0.2754,1,weekly,13.71,2023,6,...,1,0,0,0,1,0,0,1,-0.35,NaN
3288061,4255,2023-05-21,chain,134.00,0.3057,1,weekly,93.04,2023,5,...,0,0,0,0,0,0,0,0,NaN,NaN
3288062,4255,2023-05-21,chain,134.00,0.3057,1,weekly,93.04,2023,5,...,0,0,0,0,0,0,0,0,NaN,NaN


In [66]:

# Avg Final Price by Competitor Over Time (Same SKUs Only)
sku_counts = df3.groupby(['time_key', 'sku'])['competitor'].nunique().reset_index(name='comp_count')

valid_skus = sku_counts[sku_counts['comp_count'] == df3['competitor'].nunique()]
df_common = df3.merge(valid_skus[['time_key', 'sku']], on=['time_key', 'sku'], how='inner')

price_trend_common = df_common.groupby(['time_key', 'competitor'])['pvp_final'].mean().reset_index()

fig = px.line(price_trend_common, x='time_key', y='pvp_final', color='competitor',
              title='Avg Final Price by Competitor Over Time (Same SKUs Only)',
              labels={'pvp_final': 'Avg Final Price', 'time_key': 'Date'})
fig.show()


# Average discount by competitor
discount_avg = df3.groupby('competitor')['discount'].mean().reset_index()
fig2 = px.bar(discount_avg, x='competitor', y='discount',
              title='Average Discount by Competitor',
              labels={'discount': 'Average Discount'})
fig2.show()

# Promo frequency by day of week
promo_day = df3.groupby('day_of_week')['flag_promo'].mean().reset_index()
fig3 = px.bar(promo_day, x='day_of_week', y='flag_promo',
              title='Promotion Frequency by Day of the Week',
              labels={'flag_promo': 'Promotion Rate', 'day_of_week': 'Day of Week'})
fig3.show()

# Price difference (competitorA vs chain) by month
diff_by_month = df3[df3['competitor'] == 'competitorA'].groupby('month')['diff_A'].mean().reset_index()
fig4 = px.bar(diff_by_month, x='month', y='diff_A',
              title='Average Final Price Difference (CompetitorA - Chain) by Month',
              labels={'diff_A': 'Price Difference'})
fig4.show()

# Price difference (competitorB vs chain) by month
diff_by_month = df3[df3['competitor'] == 'competitorB'].groupby('month')['diff_B'].mean().reset_index()
fig4 = px.bar(diff_by_month, x='month', y='diff_B',
              title='Average Final Price Difference (CompetitorB - Chain) by Month',
              labels={'diff_B': 'Price Difference'})
fig4.show()

# % of products on promo during seasonal events
seasonal_flags = ['is_christmas_season', 'is_black_friday', 'is_summer', 'is_back_to_school']
promo_stats = []

for event in seasonal_flags:
    total = df3[df3[event] == 1].shape[0]
    on_promo = df3[(df3[event] == 1) & (df3['flag_promo'] == 1)].shape[0]
    promo_stats.append({
        'Event': event.replace('is_', '').replace('_', ' ').title(),
        'Promo Rate (%)': (on_promo / total * 100) if total > 0 else 0
    })

promo_df = pd.DataFrame(promo_stats)
fig5 = px.bar(promo_df, x='Event', y='Promo Rate (%)',
              title='Promo Rate During Seasonal Events')
fig5.show()

# Average price during vs outside seasonal events
seasonal_price = []

for event in seasonal_flags:
    price_during = df3[df3[event] == 1]['pvp_final'].mean()
    price_outside = df3[df3[event] == 0]['pvp_final'].mean()
    label = event.replace('is_', '').replace('_', ' ').title()
    seasonal_price.append({'Event': label, 'Period': 'During', 'Average Price': price_during})
    seasonal_price.append({'Event': label, 'Period': 'Outside', 'Average Price': price_outside})

price_df = pd.DataFrame(seasonal_price)
fig6 = px.bar(price_df, x='Event', y='Average Price', color='Period',
              barmode='group', title='Average Final Price During vs Outside Seasonal Events')
fig6.show()




In [68]:
# Objective: Analyze the monthly price growth rates of SKUs that are common 
# across all competitors in each month. This involves identifying SKUs 
# shared by all competitors monthly, calculating their average prices per 
# competitor, computing month-over-month growth rates, and visualizing these trends.

# Ensure datetime format and create year-month column
df3['time_key'] = pd.to_datetime(df3['time_key'])
df3['year_month'] = df3['time_key'].dt.to_period('M').astype(str)

# Find SKUs common to all competitors each month
sku_month_counts = df3.groupby(['year_month', 'sku'])['competitor'].nunique().reset_index(name='comp_count')
num_competitors = df3['competitor'].nunique()
common_skus_by_month = sku_month_counts[sku_month_counts['comp_count'] == num_competitors]

# Filter df3 to keep only SKUs common to all competitors each month
df_common_month = df3.merge(common_skus_by_month[['year_month', 'sku']], on=['year_month', 'sku'], how='inner')

# Calculate monthly average price by competitor
monthly_avg = df_common_month.groupby(['year_month', 'competitor'])['pvp_final'].mean().reset_index()

# Calculate month-over-month growth rate
monthly_avg = monthly_avg.sort_values(['competitor', 'year_month'])
monthly_avg['growth_rate'] = monthly_avg.groupby('competitor')['pvp_final'].pct_change() * 100

# Plot the growth rates
fig = px.line(monthly_avg.dropna(), x='year_month', y='growth_rate', color='competitor',
              title='Monthly Price Growth Rate (Only Common SKUs per Month)',
              labels={'year_month': 'Month', 'growth_rate': 'Growth Rate (%)'},
              markers=True)

fig.update_layout(xaxis_tickangle=45)
fig.show()




In [69]:
# This code reads product structure information in chunks to efficiently map SKUsto their corresponding structure levels (level 2 and level 4). 
# The mappings are then applied to the main dataframe (df3), enriching it with hierarchical category information for subsequent analysis.

df1_path = 'product_structures_sales.csv'

sku_structure_map_2 = {}
sku_structure_map_4 = {}
chunksize = 100_000

for chunk in pd.read_csv(df1_path, usecols=['sku', 'structure_level_2', 'structure_level_4'], chunksize=chunksize):
    chunk = chunk.drop_duplicates(subset='sku')
    sku_structure_map_2.update(chunk.set_index('sku')['structure_level_2'].to_dict())
    sku_structure_map_4.update(chunk.set_index('sku')['structure_level_4'].to_dict())

df3['structure_level_2'] = df3['sku'].map(sku_structure_map_2)
df3['structure_level_4'] = df3['sku'].map(sku_structure_map_4)



In [70]:
df3.columns

Index(['sku', 'time_key', 'competitor', 'pvp_was', 'discount', 'flag_promo',
       'leaflet', 'pvp_final', 'year', 'month', 'day_of_week', 'week',
       'is_weekend', 'is_christmas_season', 'is_new_year', 'is_summer',
       'is_back_to_school', 'is_black_friday', 'promo_in_christmas',
       'promo_in_summer', 'promo_in_back_to_school', 'promo_in_black_friday',
       'is_seasonal_promo', 'diff_A', 'diff_B', 'year_month',
       'structure_level_2', 'structure_level_4'],
      dtype='object')

In [99]:
# Analyze which competitor tends to be cheaper or more expensive by category, 
# considering data aggregated up to structure level 2.


# Calculate the average price per competitor and category (structure_level_2)
avg_price_by_comp_cat = df3.groupby(['structure_level_2', 'competitor'])['pvp_final'].mean().reset_index()

# Pivot to get competitors as columns
pivot_avg = avg_price_by_comp_cat.pivot(index='structure_level_2', columns='competitor', values='pvp_final')

# Select only numeric competitor columns (prices)
competitor_cols = pivot_avg.select_dtypes(include='number').columns

# Find cheapest and most expensive competitors per category (structure_level_2)
pivot_avg['cheapest_competitor'] = pivot_avg[competitor_cols].idxmin(axis=1)
pivot_avg['most_expensive_competitor'] = pivot_avg[competitor_cols].idxmax(axis=1)

result = pivot_avg.reset_index()

summary = result[['structure_level_2', 'cheapest_competitor', 'most_expensive_competitor']]
summary





competitor,structure_level_2,cheapest_competitor,most_expensive_competitor
0,101.0,competitorB,chain
1,102.0,competitorB,competitorA
2,103.0,competitorB,competitorA
3,104.0,competitorB,competitorA
4,105.0,competitorB,chain
5,106.0,competitorB,chain
6,201.0,competitorB,competitorA
7,202.0,competitorB,chain
8,301.0,competitorB,competitorA
9,302.0,competitorB,chain


In [77]:
# Objective: Calculate the average discount per competitor across product categories up to structure level 2.
# The discount is defined as the difference between the chain price and the competitor's price, 
# where a positive value indicates the competitor is cheaper.


# Group by competitor and structure_level_2, calculate average discount
avg_discount_by_comp_cat = df3.groupby(['structure_level_2', 'competitor'])['discount'].mean().reset_index()

# Rename column for clarity
avg_discount_by_comp_cat = avg_discount_by_comp_cat.rename(columns={'discount': 'avg_discount'})

pivot_discount = avg_discount_by_comp_cat.pivot(index='structure_level_2', columns='competitor', values='avg_discount')
pivot_discount


competitor,chain,competitorA,competitorB
structure_level_2,,,
101.0,0.014329,0.042124,0.024602
102.0,0.007890,0.014301,0.018636
103.0,0.017523,0.020133,0.008602
104.0,0.002357,0.002475,0.014265
105.0,0.003644,0.003207,0.010836
106.0,0.006029,0.011241,0.003977
201.0,0.206327,0.046415,0.065585
202.0,0.085705,0.075879,0.025063
301.0,0.094702,0.051447,0.021387


In [95]:
# Objective: Analyze how prices vary depending on promotional strategies,
# using the 'flag_promo' column (0 = no promotion, 1 = promotion active),
# across competitors and product categories (structure_level_2).

# Step 1: Group and calculate average price, standard deviation, and count
price_variation = df3.groupby(['competitor', 'structure_level_2', 'flag_promo'])['pvp_final'].agg(
    avg_price='mean',
    price_std='std',
    count='count'
).reset_index()

# Step 2: Pivot to compare prices with and without promotion
pivot_avg = price_variation.pivot_table(
    index=['competitor', 'structure_level_2'],
    columns='flag_promo',
    values='avg_price'
).rename(columns={0: 'no_promo_avg_price', 1: 'promo_avg_price'}).reset_index()

pivot_std = price_variation.pivot_table(
    index=['competitor', 'structure_level_2'],
    columns='flag_promo',
    values='price_std'
).rename(columns={0: 'no_promo_std', 1: 'promo_std'}).reset_index()

# Step 3: Merge average and std pivot tables
variation_summary = pivot_avg.merge(pivot_std, on=['competitor', 'structure_level_2'])

# Step 4: Reshape for plotting
variation_melted = variation_summary.melt(
    id_vars=['competitor', 'structure_level_2'],
    value_vars=['no_promo_avg_price', 'promo_avg_price'],
    var_name='Promotion_Status',
    value_name='Average_Price'
)

# Step 5: Clean labels
variation_melted['Promotion_Status'] = variation_melted['Promotion_Status'].map({
    'no_promo_avg_price': 'No Promo',
    'promo_avg_price': 'Promo'
})

# Step 6: Plot results
variation_melted['structure_level_2'] = variation_melted['structure_level_2'].astype(str)
variation_melted['structure_level_2'] = pd.Categorical(
    variation_melted['structure_level_2'],
    categories=variation_melted['structure_level_2'].unique(),
    ordered=True
)

fig = px.bar(
    variation_melted,
    x='structure_level_2',
    y='Average_Price',
    color='Promotion_Status',
    barmode='group',
    facet_col='competitor',
    facet_col_wrap=1,
    title='Average Price with vs without Promotion per Category and Competitor',
    labels={
        'structure_level_2': 'Category (Level 2)',
        'Average_Price': 'Average Price'
    },
    height=300 * variation_melted['competitor'].nunique(),
    width=1200
)

fig.update_layout(
    xaxis_tickangle=45,
    margin=dict(t=50, b=50, l=40, r=40)
)

fig.show()


In [90]:
test_results_mw = []

grouped = df3.groupby(['competitor', 'structure_level_2'])

for (competitor, category), group in grouped:
    promo_prices = group[group['flag_promo'] == 1]['pvp_final'].dropna()
    no_promo_prices = group[group['flag_promo'] == 0]['pvp_final'].dropna()

    if len(promo_prices) > 1 and len(no_promo_prices) > 1:
        # Mann-Whitney test, alternative='two-sided' para teste bilateral
        stat, p_val = mannwhitneyu(promo_prices, no_promo_prices, alternative='two-sided')

        test_results_mw.append({
            'Competitor': competitor,
            'Category': category,
            'Promo Median': promo_prices.median(),
            'No Promo Median': no_promo_prices.median(),
            'MW-Statistic': stat,
            'P-Value': p_val,
            'Significant': p_val < 0.05
        })

df_tests_mw = pd.DataFrame(test_results_mw)
print(df_tests_mw[df_tests_mw['Significant'] == True].sort_values('P-Value'))





     Competitor  Category  Promo Median  No Promo Median  MW-Statistic  \
11        chain     304.0         27.60            39.75  1.206182e+09   
21  competitorA     301.0         25.86            34.54  2.579553e+08   
20  competitorA     202.0         22.39            31.94  2.415714e+09   
24  competitorA     304.0         29.33            39.75  4.502621e+08   
14  competitorA     102.0         27.60            39.23  3.998258e+07   
13  competitorA     101.0         34.54            43.91  1.376925e+08   
7         chain     202.0         25.86            34.54  7.893028e+09   
8         chain     301.0         27.60            34.54  9.205078e+08   
9         chain     302.0         34.54            39.75  2.366992e+10   
10        chain     303.0         31.24            34.54  9.987675e+09   
22  competitorA     302.0         32.28            38.88  7.464268e+09   
34  competitorB     301.0         20.65            25.17  8.774006e+06   
23  competitorA     303.0         33.6

In [107]:
colors = {
    'chain': ('#1f77b4', '#aec7e8'),       
    'competitorA': ('#2ca02c', '#98df8a'),  
    'competitorB': ('#ff7f0e', '#ffbb78')  
}


fig = go.Figure()

for competitor in df_tests_mw['Competitor'].unique():
    df_comp = df_tests_mw[df_tests_mw['Competitor'] == competitor]
    promo_color, no_promo_color = colors.get(competitor, ('#000000', '#888888')) 
    
    # Promo bars
    fig.add_trace(go.Bar(
        x=df_comp['Category'].astype(str),
        y=df_comp['Promo Median'],
        name=f'{competitor} - Promo',
        marker_color=promo_color,
        offsetgroup=competitor,
        legendgroup=competitor,
    ))
    
    # No promo bars
    fig.add_trace(go.Bar(
        x=df_comp['Category'].astype(str),
        y=df_comp['No Promo Median'],
        name=f'{competitor} - No Promo',
        marker_color=no_promo_color,
        offsetgroup=competitor,
        legendgroup=competitor,
        opacity=0.7
    ))

fig.update_layout(
    title='Median Price Comparison: Promotion vs No Promotion by Category and Competitor',
    xaxis_title='Category (structure_level_2)',
    yaxis_title='Median Price',
    barmode='group',
    xaxis={'categoryorder': 'category ascending'},
    legend_title_text='Competitor & Promo Status',
    width=900,
    height=500
)

fig.show()




## Merging Datasets

In [140]:
df_merged = df3.merge(
    df2,
    left_on=['time_key', 'competitor'], 
    right_on=['start_date', 'competitor'], 
    how='left', 
    indicator=True
)


In [141]:
print(df_merged['_merge'].value_counts())
df_merged = df_merged.drop(columns=['_merge'])

_merge
left_only     3066169
both           223944
right_only          0
Name: count, dtype: int64


In [142]:
df_merged.loc[(df_merged['flag_promo'] == 0) & (df_merged['discount'] != 0), 'flag_promo'] = 1

In [143]:
df_merged['chain_campaign'] = df_merged['chain_campaign'].where(df_merged['flag_promo'] == 1, 'no_campaign')

In [144]:
df_merged['duration_days'] = df_merged['duration_days'].where(df_merged['flag_promo'] == 1, 0)

In [145]:
df_merged.columns

Index(['sku', 'time_key', 'competitor', 'pvp_was', 'discount', 'flag_promo',
       'leaflet', 'pvp_final', 'year', 'month', 'day_of_week', 'week',
       'is_weekend', 'is_christmas_season', 'is_new_year', 'is_summer',
       'is_back_to_school', 'is_black_friday', 'promo_in_christmas',
       'promo_in_summer', 'promo_in_back_to_school', 'promo_in_black_friday',
       'is_seasonal_promo', 'diff_A', 'diff_B', 'year_month',
       'structure_level_2', 'structure_level_4', 'start_date', 'end_date',
       'chain_campaign', 'duration_days', 'start_day'],
      dtype='object')

In [146]:
columns_to_drop= ['start_date', 'end_date','start_day', 'week']
df_merged = df_merged.drop (columns = columns_to_drop)

In [147]:
df_merged

,sku,time_key,competitor,pvp_was,discount,flag_promo,leaflet,pvp_final,year,month,...,promo_in_back_to_school,promo_in_black_friday,is_seasonal_promo,diff_A,diff_B,year_month,structure_level_2,structure_level_4,chain_campaign,duration_days
0,2056,2024-03-12,chain,21.70,0.0000,0,NaN,21.70,2024,3,...,0,0,0,NaN,NaN,2024-03,302.0,3021101.0,no_campaign,0.0
1,4435,2023-06-21,chain,18.92,0.2754,1,weekly,13.71,2023,6,...,0,0,1,NaN,NaN,2023-06,303.0,3030708.0,NaN,NaN
2,1951,2023-03-03,competitorA,60.58,0.3524,1,NaN,39.23,2023,3,...,0,0,0,-21.35,NaN,2023-03,302.0,3020806.0,A1,2.0
3,2135,2024-06-05,chain,55.37,0.2509,1,weekly,41.48,2024,6,...,0,0,1,NaN,NaN,2024-06,301.0,3010801.0,NaN,NaN
4,3949,2023-05-29,chain,51.90,0.0000,0,NaN,51.90,2023,5,...,0,0,0,NaN,NaN,2023-05,101.0,1010407.0,no_campaign,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3290108,4726,2024-06-08,chain,70.99,0.5158,1,weekly,34.37,2024,6,...,0,0,1,NaN,NaN,2024-06,303.0,3030808.0,NaN,NaN
3290109,4692,2023-06-26,competitorA,18.92,0.2754,1,weekly,13.71,2023,6,...,0,0,1,-0.35,NaN,2023-06,302.0,3020601.0,NaN,NaN
3290110,4255,2023-05-21,chain,134.00,0.3057,1,weekly,93.04,2023,5,...,0,0,0,NaN,NaN,2023-05,201.0,2010806.0,NaN,NaN
3290111,4255,2023-05-21,chain,134.00,0.3057,1,weekly,93.04,2023,5,...,0,0,0,NaN,NaN,2023-05,201.0,2010806.0,NaN,NaN


In [148]:
df_merged.columns

Index(['sku', 'time_key', 'competitor', 'pvp_was', 'discount', 'flag_promo',
       'leaflet', 'pvp_final', 'year', 'month', 'day_of_week', 'is_weekend',
       'is_christmas_season', 'is_new_year', 'is_summer', 'is_back_to_school',
       'is_black_friday', 'promo_in_christmas', 'promo_in_summer',
       'promo_in_back_to_school', 'promo_in_black_friday', 'is_seasonal_promo',
       'diff_A', 'diff_B', 'year_month', 'structure_level_2',
       'structure_level_4', 'chain_campaign', 'duration_days'],
      dtype='object')

In [149]:
df_merged_index = df_merged.set_index(['time_key', 'sku'])
df_merged_index= df_merged_index.sort_index()

In [150]:
df_merged_index.head(10)

competitor  pvp_was  discount  flag_promo leaflet  \
time_key   sku                                                        
2023-01-03 1615        chain    39.75    0.0000           0     NaN   
           1618        chain    22.39    0.0000           0     NaN   
           1624        chain    16.49    0.0000           0     NaN   
           1625        chain    16.49    0.0000           0     NaN   
           1626  competitorA    27.60    0.3145           1     NaN   
           1626        chain    27.60    0.2518           1  weekly   
           1627  competitorA    27.60    0.3145           1     NaN   
           1627        chain    27.60    0.2518           1  weekly   
           1630  competitorA    27.60    0.0000           0     NaN   
           1630        chain    27.60    0.0000           0     NaN   

                 pvp_final  year  month  day_of_week  is_weekend  ...  \
time_key   sku                                                    ...   
2023-01-03 1615      39.75  2023      1            1           0  ...   
           1618      22.39  2023      1            1           0  ...   
           1624      16.49  2023      1            1           0  ...   
           1625      16.49  2023      1            1           0  ...   
           1626      18.92  2023      1            1           0  ...   
           1626      20.65  2023      1            1           0  ...   
           1627      18.92  2023      1            1           0  ...   
           1627      20.65  2023      1            1           0  ...   
           1630      27.60  2023      1            1           0  ...   
           1630      27.60  2023      1            1           0  ...   

                 promo_in_back_to_school  promo_in_black_friday  \
time_key   sku                                                    
2023-01-03 1615                        0                      0   
           1618                        0                      0   
           1624                        0                      0   
           1625                        0                      0   
           1626                        0                      0   
           1626                        0                      0   
           1627                        0                      0   
           1627                        0                      0   
           1630                        0                      0   
           1630                        0                      0   

                 is_seasonal_promo  diff_A  diff_B  year_month  \
time_key   sku                                                   
2023-01-03 1615                  0     NaN     NaN     2023-01   
           1618                  0     NaN     NaN     2023-01   
           1624                  0     NaN     NaN     2023-01   
           1625                  0     NaN     NaN     2023-01   
           1626                  0   -1.73     NaN     2023-01   
           1626                  0     NaN     NaN     2023-01   
           1627                  0   -1.73     NaN     2023-01   
           1627                  0     NaN     NaN     2023-01   
           1630                  0    0.00     NaN     2023-01   
           1630                  0     NaN     NaN     2023-01   

                 structure_level_2  structure_level_4  chain_campaign  \
time_key   sku                                                          
2023-01-03 1615              101.0          1010308.0     no_campaign   
           1618              202.0          2020601.0     no_campaign   
           1624              305.0          3050814.0     no_campaign   
           1625              305.0          3050814.0     no_campaign   
           1626              202.0          2020607.0             NaN   
           1626              202.0          2020607.0              C1   
           1627              202.0          2020607.0             NaN   
           1627              202.

In [151]:
# Objective: Analyze promotional discounts by calculating the average discount per campaign type.

# Filter only records where flag_promo is equal to 1
df_promo = df_merged[df_merged['flag_promo'] == 1]

# Calculate average discount by type of campaign
average_discount = df_promo.groupby('chain_campaign')['discount'].mean().reset_index()

# Sort (optional, to see from highest to lowest)
average_discount = average_discount.sort_values(by='discount', ascending=False)

# View result
print(average_discount)


  chain_campaign  discount
4             C2  0.276559
3             C1  0.273320
0             A1  0.269657
1             A2  0.267069
2             A3  0.262710


In [120]:
# Objective: Perform competitor analysis by counting absences of 'chain', 'competitorA', and 'competitorB' in the DataFrame.

df = df_merged.copy()

# Filter data with prices from 'chain', 'competitorA', and 'competitorB'
competitors = ['chain', 'competitorA', 'competitorB']
df_filtered = df[df['competitor'].isin(competitors)]

# List of expected competitors
expected_competitors = ['chain', 'competitorA', 'competitorB']

# Group data by time_key and sku, and count present competitors
grouped = df_filtered.groupby(['time_key', 'sku'])['competitor'].apply(set)

# Create a dictionary to count absences
absence_count = {competitor: 0 for competitor in expected_competitors}

# Check absences for each combination of time_key and sku
for idx, competitors in grouped.items():
    missing = set(expected_competitors) - competitors
    for competitor in missing:
        absence_count[competitor] += 1

absence_count




{'chain': 0, 'competitorA': 608594, 'competitorB': 1421703}

# 🧾 Imputation of Missing Competitor Prices

## 🎯 Objective

Ensure that for every combination of **`(time_key, sku)`**, we have price data (`pvp_final`) for all three entities:
- `chain` (always present)
- `competitorA`
- `competitorB`

## ⚙️ Approach

- The dataset contains full pricing data for `chain`, but many competitor prices are missing.
- We leverage the real price from `chain`, along with several engineered features (e.g., calendar events, promo flags, seasonality), to **train a supervised machine learning model** that predicts missing competitor prices.
- The model learns from historical data where competitor prices are available and generalizes to estimate prices where they are missing.

## ✅ Outcome

- A complete dataset with one row per `(time_key, sku, competitor)` including:
  - Real prices for `chain`
  - Real or imputed prices for `competitorA` and `competitorB`
  - A column indicating whether the price is **real or imputed**

This dataset can now be reliably used for **competitive analysis**, **price optimization**, or further modeling.


In [79]:
a

NameError: name 'a' is not defined

In [124]:
# 📌 STEP 1 — Ensure full structure: one row per competitor per date and SKU

# 1. Get all unique time_key + sku combinations
time_sku_pairs = df_merged[['time_key', 'sku']].drop_duplicates()

# 2. Cross join with all 3 competitors
full_index = time_sku_pairs.assign(key=1).merge(
    pd.DataFrame({'competitor': ['chain', 'competitorA', 'competitorB'], 'key': 1}),
    on='key'
).drop('key', axis=1)

# 3. Left join with original dataset to fill missing rows
df_full = full_index.merge(df_merged, on=['time_key', 'sku', 'competitor'], how='left')

# 📌 STEP 2 — Merge real chain prices to all rows (used as a feature)
chain_prices = df_merged[df_merged['competitor'] == 'chain'][['time_key', 'sku', 'pvp_final']]
chain_prices = chain_prices.rename(columns={'pvp_final': 'chain_price'})

df_full = df_full.merge(chain_prices, on=['time_key', 'sku'], how='left')

# 📌 STEP 3 — Add historical price differences and group-level statistics

# Sort by SKU and date for temporal operations
df_full = df_full.sort_values(by=['sku', 'time_key'])

# Helper functions for expanding statistics (excluding current row)
def expanding_shifted_mean(x):
    return x.shift(1).expanding().mean()

def expanding_shifted_std(x):
    return x.shift(1).expanding().std()

# Compute group-level rolling statistics
df_full['diffA_mean_level4'] = (
    df_full.groupby(['sku', 'structure_level_4'])['diff_A']
    .transform(expanding_shifted_mean)
)
df_full['diffB_mean_level4'] = (
    df_full.groupby(['sku', 'structure_level_4'])['diff_B']
    .transform(expanding_shifted_mean)
)
df_full['diffA_std_level4'] = (
    df_full.groupby(['sku', 'structure_level_4'])['diff_A']
    .transform(expanding_shifted_std)
)
df_full['diffB_std_level4'] = (
    df_full.groupby(['sku', 'structure_level_4'])['diff_B']
    .transform(expanding_shifted_std)
)

# Create lag features
df_full['lag_diffA'] = df_full.groupby('sku')['diff_A'].shift(1)
df_full['lag_diffB'] = df_full.groupby('sku')['diff_B'].shift(1)

# 📌 STEP 4 — Filter training candidates (only rows with known competitor prices)
df_competitors = df_full[
    df_full['competitor'].isin(['competitorA', 'competitorB']) &
    df_full['pvp_final'].notnull()
]

# 📌 STEP 5 — Define features and target
features = [
    'chain_price','discount', 'flag_promo', 'year', 'month', 'day_of_week',
    'is_weekend', 'is_christmas_season', 'is_new_year', 'is_summer',
    'is_back_to_school', 'is_black_friday', 'promo_in_christmas',
    'promo_in_summer', 'promo_in_back_to_school', 'promo_in_black_friday',
    'is_seasonal_promo', 'structure_level_4', 'chain_campaign', 'duration_days',
    'competitor', 'lag_diffA', 'lag_diffB', 
    'diffA_mean_level4', 'diffB_mean_level4', 'diffA_std_level4', 'diffB_std_level4'
]

target = 'pvp_final'

# 📌 STEP 6 — Train-test split: grouped by time_key + sku
# 1. Get unique time_key + sku pairs
unique_time_sku = df_competitors[['time_key', 'sku']].drop_duplicates()

# 2. Split these pairs into train and test sets
time_sku_train, time_sku_test = train_test_split(
    unique_time_sku, test_size=0.2, random_state=42
)

# 3. Merge to get complete rows for each group
df_train_split = df_competitors.merge(time_sku_train, on=['time_key', 'sku'])
df_test_split  = df_competitors.merge(time_sku_test, on=['time_key', 'sku'])

# 4. Prepare feature and target datasets
X_train = df_train_split[features]
y_train = df_train_split[target]

X_test = df_test_split[features]
y_test = df_test_split[target]


In [126]:
# 🧾 STEP 5.1 — Train the Model (Linear Regression)

# Define categorical and numerical columns
categorical = ['structure_level_4', 'chain_campaign', 'competitor']
numerical = [col for col in features if col not in categorical]

# Preprocessing pipeline
preprocessor = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), numerical),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical)
])

# Final pipeline
pipeline = Pipeline([
    ('prep', preprocessor),
    ('model', LinearRegression())
])

# Fit the model
pipeline.fit(X_train, y_train)


with open('pipeline_Lr.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [127]:
# Create a full copy of the test set (with all features)
X_test_masked = X_test.copy()

# Predict imputed values using the trained model
y_pred = pipeline.predict(X_test_masked)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("🔍 Evaluation on Masked Test Set:")
print(f"MAE:  {mae:.4f}")
print(f"R²:   {r2:.4f}")




🔍 Evaluation on Masked Test Set:
MAE:  4.2656
R²:   0.9787


# RANDOM FOREST

In [128]:
# 🧾 STEP 5.2— Train the Model (Random Forest Regression)

# Define categorical and numerical columns
categorical = ['structure_level_4', 'chain_campaign', 'competitor']
numerical = [col for col in features if col not in categorical]

# Preprocessing pipeline
preprocessor = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), numerical),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical)
])

# Final pipeline with Random Forest
pipeline = Pipeline([
    ('prep', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Fit the model
pipeline.fit(X_train, y_train)


with open('pipeline_rf.pkl', 'wb') as f:
    pickle.dump(pipeline, f)


KeyboardInterrupt: 

In [ ]:
# 🧾 STEP 6.2— Mask pvp_final in test set (simulate missing competitor prices)

# Create a copy of X_test to avoid modifying original
X_test_masked_rf = X_test.copy()

# Simular falta de dados nas variáveis de defasagem
X_test_masked_rf[['lag_diffA', 'lag_diffB']] = np.nan

# Use the trained pipeline to predict on masked test data
y_pred = pipeline.predict(X_test_masked_rf)

# 🧾 STEP 7 — Evaluate the imputation
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("🔍 Evaluation on Masked Test Set:")
print(f"MAE:  {mae:.4f}")
print(f"R²:   {r2:.4f}")

In [129]:
# Load Pipeline
with open("pipeline_rf.pkl", "rb") as f:
    pipeline = pickle.load(f)


In [130]:
# 🧾 STEP 7 — Predict Missing Prices

# Identify rows to impute (missing competitor prices only)
df_missing = df_full[
    df_full['competitor'].isin(['competitorA', 'competitorB']) &
    df_full['pvp_final'].isnull()
].copy()

# Prepare input features for prediction
X_missing = df_missing[features]

# Predict missing prices using trained model
predicted_prices = pipeline.predict(X_missing)

# Assign predictions back to the original DataFrame
df_full.loc[df_missing.index, 'pvp_final'] = predicted_prices
df_full.loc[df_missing.index, 'is_imputed'] = 1
df_full.loc[df_missing.index, 'pvp_final_source'] = 'model_rf_based_on_chain_price'


In [87]:
df_full.columns

Index(['time_key', 'sku', 'competitor', 'pvp_was', 'discount', 'flag_promo',
       'leaflet', 'pvp_final', 'year', 'month', 'day_of_week', 'is_weekend',
       'is_christmas_season', 'is_new_year', 'is_summer', 'is_back_to_school',
       'is_black_friday', 'promo_in_christmas', 'promo_in_summer',
       'promo_in_back_to_school', 'promo_in_black_friday', 'is_seasonal_promo',
       'diff_A', 'diff_B', 'year_month', 'structure_level_4', 'chain_campaign',
       'duration_days', 'chain_price', 'diffA_mean_level4',
       'diffB_mean_level4', 'diffA_std_level4', 'diffB_std_level4',
       'lag_diffA', 'lag_diffB', 'is_imputed', 'pvp_final_source'],
      dtype='object')

In [ ]:
# Objective: Select specific columns from the full DataFrame and save them to a CSV file.

# Select only the columns 'A' and 'B'
columns_to_dataframe = df_full[['time_key', 'sku', 'competitor', 'pvp_final', 'structure_level_4', 'flag_promo', 'chain_price']]

# Save the selected columns to a CSV file
columns_to_dataframe.to_csv('df_final.csv', index=False)


In [132]:
pd.crosstab(df_merged['chain_campaign'], df_merged['is_seasonal_promo'])

is_seasonal_promo,0,1
chain_campaign,,
A1,2173,2257
A2,6375,2255
A3,2281,1447
C1,15132,11793
C2,12173,7468
no_campaign,2380611,0


In [133]:
competitors = df_merged[df_merged['competitor'].isin(['competitorA', 'competitorB'])]
competitors.head(10)

,sku,time_key,competitor,pvp_was,discount,flag_promo,leaflet,pvp_final,year,month,...,promo_in_back_to_school,promo_in_black_friday,is_seasonal_promo,diff_A,diff_B,year_month,structure_level_2,structure_level_4,chain_campaign,duration_days
2,1951,2023-03-03,competitorA,60.58,0.3524,1,NaN,39.23,2023,3,...,0,0,0,-21.35,NaN,2023-03,302.0,3020806.0,A1,2.0
5,1683,2023-11-26,competitorA,62.14,0.1117,1,NaN,55.20,2023,11,...,0,0,0,-5.38,NaN,2023-11,202.0,2020810.0,NaN,NaN
8,2309,2023-12-20,competitorA,77.93,0.0000,0,NaN,77.93,2023,12,...,0,0,0,19.44,NaN,2023-12,302.0,3020313.0,no_campaign,0.0
11,4068,2023-03-05,competitorA,27.60,0.0000,0,NaN,27.60,2023,3,...,0,0,0,0.00,NaN,2023-03,302.0,3020406.0,no_campaign,0.0
12,3904,2023-08-16,competitorA,26.73,0.0000,0,NaN,26.73,2023,8,...,0,0,0,0.87,NaN,2023-08,201.0,2011101.0,no_campaign,0.0
13,2966,2023-10-16,competitorB,19.96,0.0000,0,NaN,19.96,2023,10,...,0,0,0,NaN,-1.74,2023-10,106.0,1060414.0,no_campaign,0.0
14,4654,2023-09-02,competitorA,25.86,0.4026,1,short,15.45,2023,9,...,1,0,1,-10.41,NaN,2023-09,202.0,2020607.0,NaN,NaN
16,4548,2024-07-29,competitorA,20.65,0.1680,1,NaN,17.18,2024,7,...,0,0,1,-8.68,NaN,2024-07,101.0,1010708.0,NaN,NaN
20,1675,2024-08-04,competitorA,21.52,0.0000,0,NaN,21.52,2024,8,...,0,0,0,0.00,NaN,2024-08,302.0,3020608.0,no_campaign,0.0
21,3705,2023-04-16,competitorA,50.16,0.2596,1,weekly,37.14,2023,4,...,0,0,0,0.00,NaN,2023-04,202.0,2020609.0,NaN,NaN


In [134]:
competitorsB = df_merged[df_merged['competitor'].isin(['competitorB'])]
competitorsB

,sku,time_key,competitor,pvp_was,discount,flag_promo,leaflet,pvp_final,year,month,...,promo_in_back_to_school,promo_in_black_friday,is_seasonal_promo,diff_A,diff_B,year_month,structure_level_2,structure_level_4,chain_campaign,duration_days
13,2966,2023-10-16,competitorB,19.96,0.0000,0,NaN,19.96,2023,10,...,0,0,0,NaN,-1.74,2023-10,106.0,1060414.0,no_campaign,0.0
25,4467,2023-07-11,competitorB,24.13,0.0000,0,NaN,24.13,2023,7,...,0,0,0,NaN,0.00,2023-07,103.0,1031109.0,no_campaign,0.0
70,1655,2023-09-02,competitorB,19.44,0.0000,0,NaN,19.44,2023,9,...,0,0,0,NaN,1.74,2023-09,303.0,3031106.0,no_campaign,0.0
84,4563,2023-02-16,competitorB,84.88,0.0000,0,NaN,84.88,2023,2,...,0,0,0,NaN,6.95,2023-02,101.0,1010409.0,no_campaign,0.0
98,4564,2024-03-17,competitorB,24.99,0.0000,0,NaN,24.99,2024,3,...,0,0,0,NaN,-12.85,2024-03,303.0,3031106.0,no_campaign,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3290069,2180,2023-09-21,competitorB,19.44,0.2052,1,weekly,15.45,2023,9,...,1,0,1,NaN,-3.47,2023-09,105.0,1050306.0,NaN,NaN
3290070,2180,2023-09-21,competitorB,19.44,0.2052,1,weekly,15.45,2023,9,...,1,0,1,NaN,-3.47,2023-09,105.0,1050306.0,NaN,NaN
3290083,3310,2023-07-06,competitorB,86.61,0.0000,0,NaN,86.61,2023,7,...,0,0,0,NaN,9.89,2023-07,201.0,2010807.0,no_campaign,0.0
3290097,4564,2023-10-01,competitorB,20.65,0.0000,0,NaN,20.65,2023,10,...,0,0,0,NaN,0.00,2023-10,303.0,3031106.0,no_campaign,0.0


In [135]:
print(competitors['leaflet'].value_counts())

leaflet
weekly    155009
short      14897
themed     14784
Name: count, dtype: int64


In [136]:
print(competitors['flag_promo'].value_counts())

flag_promo
0    1140927
1     276657
Name: count, dtype: int64


In [137]:
print (competitors ['diff_A'].value_counts())

diff_A
 0.00     529150
 3.47      12133
 8.68      12119
-3.47       9568
 5.21       8806
           ...  
 60.73         1
-37.66         1
 17.87         1
-33.14         1
 27.77         1
Name: count, Length: 2625, dtype: int64
